In [1]:
import AI_Frameworks_LDA
from tqdm import tqdm

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/osama.nasir@ebryx.com/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
import nltk
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# sklearn
from sklearn.preprocessing import MinMaxScaler

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
from matplotlib import pyplot as plt
import matplotlib.colors as mcolors
#%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [3]:
#df_tmp = AI_Frameworks_LDA.loadMetadata()

In [4]:
#df_tmp.to_excel("metadata_text.xlsx", engine='xlsxwriter')
#len(df_tmp)

# Preprocessing Data

In [3]:
df1 = pd.read_excel('metadata_text.xlsx')
#df = df_tmp

In [4]:
df1.head()

,Unnamed: 0,filename,text
0,0,Metadata-Goal-11.pdf,Goal 11 Make cities and human settlements in...
1,1,Metadata-Goal-10.pdf,\n\nGoal 10 Reduce inequality within...
2,2,Metadata-Goal-13.pdf,Goal 13 Take urgent action to combat...
3,3,Metadata-Goal-3.pdf,\nGoal 3 Ensure healthy lives and promote w...
4,4,Metadata-Goal-2.pdf,"Goal 2 End hunger, achieve food security..."


In [6]:
len(df1)

17

In [7]:
df2 = pd.read_excel('metadata_labels.xlsx')

In [8]:
df2.head()

,filename,sdg1,sdg2,sdg3,sdg4,sdg5,sdg6,sdg7,sdg8,sdg9,sdg10,sdg11,sdg12,sdg13,sdg14,sdg15,sdg16,sdg17
0,Metadata-Goal-11.pdf,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
1,Metadata-Goal-10.pdf,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
2,Metadata-Goal-13.pdf,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
3,Metadata-Goal-3.pdf,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Metadata-Goal-2.pdf,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [9]:
len(df2)

17

In [10]:
corpus, id2word, data_lemmatized = AI_Frameworks_LDA.preprocessData(df1)

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/home/osama.nasir@ebryx.com/anaconda3/envs/myenv/lib/python3.7/site-packages/en_core_web_sm
-->
/home/osama.nasir@ebryx.com/anaconda3/envs/myenv/lib/python3.7/site-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [11]:
print(len(corpus))
print(len(id2word))
print(len(data_lemmatized))
#print(data_lemmatized)

17
3070
17


In [13]:
vocab = list()
for i in range(len(id2word)):
    vocab.append(id2word[i])

In [14]:
with open('vocab.txt', 'w') as filehandle:
    for listitem in vocab:
        filehandle.write('%s\n' % str(listitem))

# SDG model

In [12]:
#defining all 17 SDGs
sdg1 = ['africa', 'basic', 'class', 'countries', 'developing', 'disadvantaged', 'discrimination', 'distribution', 'economic', 'end', 'environment', 'equality', 'eradication', 'extreme', 'financial', 'inclusion', 'income', 'life', 'line', 'microfinance', 'poor', 'poverty', 'protection', 'quality', 'resources', 'services', 'social', 'sustainable', 'systems', 'third', 'vulnerable', 'wealth', 'world']
sdg2 = ['agriculture', 'diversity', 'food', 'genetic', 'genetics', 'growth', 'health', 'hunger', 'hungry', 'improved', 'infrastructure', 'innovations', 'legumes', 'life', 'maize', 'malnourished', 'malnutrition', 'needs', 'nourished', 'nutrition', 'nutritional', 'nutritious', 'people', 'poverty', 'produce', 'producers', 'production', 'productivity', 'quality', 'resilient', 'restrictions', 'rural', 'scale', 'seeds', 'small', 'stunted', 'stunting', 'sufficient', 'sustainable', 'trade', 'under', 'undernourished', 'wasting', 'world']
sdg3 = ['abuse', 'access', 'accidents', 'affordable', 'aids', 'air', 'alcohol', 'antenatal', 'antiretroviral', 'autonomy', 'biomedical', 'bodily', 'borne', 'care', 'child', 'clean', 'constrained', 'contamination', 'contraceptive', 'control', 'countries', 'coverage', 'death', 'deaths', 'density', 'dental', 'developing', 'disability', 'disease', 'diseases', 'drug', 'expectancy', 'family', 'health', 'healthcare', 'healthy', 'hepatitis', 'hiv', 'hygiene', 'improving', 'inclusion', 'increasing', 'indigenous', 'infected', 'international', 'life', 'lives', 'location', 'malaria', 'maternal', 'measles', 'medical', 'medicines', 'mental', 'mortality', 'narcotic', 'neonatal', 'organisation', 'planning', 'policy', 'polio', 'politics', 'pollution', 'premature', 'preventable', 'reducing', 'refugees', 'regulations', 'reproductive', 'resource', 'road', 'sanitation', 'services', 'sexual', 'soil', 'substance', 'support', 'therapy', 'tobacco', 'traffic', 'treatment', 'tuberculosis', 'universal', 'use', 'vaccines', 'violence', 'wash', 'water', 'well', 'wellbeing', 'worker', 'world']
sdg4 = ['access', 'basic', 'childhood', 'citizenship', 'cooperation', 'cultural', 'developing', 'development', 'disability', 'disparities', 'disparity', 'diversity', 'education', 'enrolment', 'equal', 'equality', 'equitable', 'equity', 'gender', 'global', 'inclusion', 'inclusive', 'innovation', 'international', 'learning', 'lifelong', 'literacy', 'numeracy', 'opportunities', 'preprimary', 'primary', 'qualified', 'refugees', 'rights', 'scholarships', 'school', 'secondary', 'sensitive', 'skills', 'sustainability', 'teacher', 'teachers', 'training', 'universal', 'vocational', 'vulnerable', 'women']
sdg5 = ['access', 'against', 'basic', 'coverage', 'dignity', 'disadvantaged', 'discrimination', 'employment', 'empower', 'empowerment', 'equal', 'equality', 'exploitation', 'female', 'feminism', 'forced', 'gender', 'genital', 'girls', 'governance', 'health', 'human', 'humanitarian', 'inclusion', 'living', 'marginalised', 'marriage', 'mutilation', 'opportunities', 'parity', 'pay', 'reproductive', 'rights', 'sexual', 'social', 'standards', 'trafficking', 'universal', 'violence', 'women', 'work', 'workplace']
sdg6 = ['access', 'accessible', 'affordable', 'aquifer', 'basins', 'cities', 'clean', 'contaminated', 'defecation', 'desalination', 'diarrhoeal', 'disasters', 'diseases', 'drinking', 'drought', 'dumping', 'ecosystem', 'ecosystems', 'efficiency', 'equitable', 'floods', 'harvesting', 'hydropower', 'hygiene', 'improving', 'inadequate', 'infrastructure', 'irrigation', 'lakes', 'latrines', 'management', 'pollution', 'protection', 'quality', 'recycled', 'resources', 'restoration', 'reuse', 'river', 'rivers', 'safe', 'sanitation', 'scarcity', 'sewerage', 'supply', 'sustainable', 'third', 'toilets', 'treatment', 'untreated', 'urban', 'waste', 'wastewater', 'water', 'water_related', 'water_use', 'withdrawals', 'world']
sdg7 = ['affordable', 'alternative', 'animal', 'battery', 'carbon', 'charcoal', 'clean', 'cleaner', 'climate', 'coal', 'economy', 'efficiency', 'electricity', 'emissions', 'energy', 'fossil', 'fossil_fuel', 'fuel', 'fuels', 'gas', 'goal', 'green', 'greenhouse', 'hydroelectric', 'infrastructure', 'modern', 'power', 'reliable', 'renewable', 'research', 'services', 'solar', 'sustainable', 'technology', 'turbine', 'vehicles', 'waste', 'wave', 'wind', 'wood']
sdg8 = ['aid', 'banking', 'child', 'consumption', 'creation', 'creativity', 'culture', 'development', 'domestic', 'economic', 'economy', 'efficiency', 'employment', 'enterprises', 'entrepreneurship', 'equal', 'eradication', 'finance', 'financial', 'forced', 'gdp', 'global', 'gross', 'growth', 'human', 'inclusive', 'innovation', 'insurance', 'job', 'jobs', 'labour', 'life', 'market', 'micro', 'migrant', 'migrants', 'modern', 'opportunities', 'oriented', 'pay', 'policies', 'policy', 'poverty', 'product', 'production', 'productive', 'productivity', 'public', 'quality', 'resource', 'rights', 'safe', 'secure', 'services', 'slavery', 'social', 'society', 'soldiers', 'stable', 'sustainable', 'tourism', 'trade', 'trafficking', 'unemployment', 'paid', 'women', 'work', 'workers', 'world', 'youth']
sdg9 = ['access', 'affordable', 'banks', 'capabilities', 'chains', 'clean', 'communication', 'cooperation', 'countries', 'credit', 'data', 'developing', 'development', 'diversification', 'economic', 'education', 'efficiency', 'electrical', 'energy', 'enterprises', 'environmentally', 'financial', 'ict', 'industrial', 'industrialisation', 'information', 'infrastructure', 'innovation', 'internet', 'irrigation', 'knowledge', 'markets', 'mobile', 'national', 'network', 'networks', 'phone', 'policy', 'power', 'public', 'quality', 'regional', 'research', 'resilient', 'resource', 'resources', 'roads', 'sanitation', 'scientific', 'security', 'service', 'services', 'society', 'sustainable', 'technological', 'technologies', 'technology', 'trade', 'transborder', 'transport', 'value', 'water']
sdg10 = ['affordable', 'age', 'ageism', 'aid', 'assistance', 'business', 'children', 'countries', 'culture', 'developing', 'development', 'disabilities', 'discrimination', 'discriminatory', 'economy', 'education', 'empower', 'equal', 'equality', 'equity', 'ethnicity', 'financial', 'foreign', 'gender', 'global', 'growth', 'health', 'homelessness', 'homophobia', 'housing', 'human', 'inclusion', 'income', 'indigenous', 'inequalities', 'inequality', 'investment', 'markets', 'migrant', 'migration', 'nations', 'opportunity', 'policy', 'population', 'poverty', 'protection', 'public', 'quality', 'race', 'racisim', 'reduce', 'religion', 'remittance', 'rights', 'rural', 'sex', 'sexism', 'social', 'society', 'states', 'trade', 'vulnerable', 'world']
sdg11 = ['adaptable', 'adaptation', 'affordable', 'air', 'buildings', 'building', 'change', 'cities', 'city', 'climate', 'communities', 'community', 'consumption', 'crowding', 'cultural', 'decentralisation', 'development', 'disaster', 'disasters', 'efficiency', 'fine', 'generation', 'green', 'growth', 'heritage', 'housing', 'human', 'impact', 'inadequate', 'informal', 'infrastructure', 'land', 'local', 'management', 'materials', 'matter', 'mitigation', 'natural', 'particulate', 'planning', 'pollution', 'population', 'public', 'quality', 'reduction', 'resilient', 'resource', 'risk', 'road', 'safe', 'safety', 'settlements', 'shanty', 'slums', 'smart', 'solid', 'spaces', 'strategy', 'suburban', 'sustainability', 'sustainable', 'systems', 'town', 'transport', 'urban', 'urbanisation', 'waste', 'water', 'water_related']
sdg12 = ['capitalism', 'cars', 'chain', 'chains', 'circular', 'commercial', 'consumer', 'consumerism', 'consumption', 'cycle', 'decarbonisation', 'development', 'distortions', 'ecological', 'economy', 'efficiency', 'efficient', 'energy', 'enterprises', 'food', 'fossil', 'fuel', 'future', 'gasses', 'generation', 'global', 'goods', 'greenhouse', 'harvest', 'industry', 'levels', 'losses', 'management', 'market', 'materialism', 'materials', 'monitoring', 'natural', 'obsolescence', 'overconsumption', 'pollution', 'practices', 'procurement', 'production', 'proof', 'public', 'recycle', 'recycling', 'reduce', 'reduction', 'renewable', 'resource', 'resources', 'responsible', 'retail', 'reuse', 'subsidies', 'supply', 'sustainable', 'tourism', 'vehicles', 'waste', 'wasteful', 'water']
sdg13 = ['action', 'adaptation', 'agreement', 'average', 'capture', 'carbon', 'change', 'changing', 'climate', 'co2', 'conversion', 'cop', 'dioxide', 'disasters', 'disease', 'economy', 'ecosystems', 'emissions', 'events', 'extreme', 'gas', 'gases', 'gender', 'global', 'greenhouse', 'hazards', 'ice', 'impact', 'infectious', 'management', 'mitigation', 'natural', 'ocean', 'paris', 'patterns', 'planning', 'policy', 'politics', 'pollution', 'refugees', 'related', 'renewable', 'resilience', 'rise', 'rising', 'sea', 'systems', 'temperature', 'warming', 'warning', 'weather']
sdg14 = ['acidification', 'areas', 'artisanal', 'biodiversity', 'bleaching', 'carbon', 'coastal', 'coastlines', 'conserve', 'coral', 'dioxide', 'ecosystem', 'ecosystems', 'fish', 'fisheries', 'fishers', 'fishing', 'global', 'grasses', 'habitats', 'illegal', 'kelp', 'law', 'management', 'marine', 'ocean', 'oceanography', 'oceans', 'overfishing', 'parks', 'policy', 'pollution', 'practices', 'productive', 'protected', 'reef', 'resources', 'sea', 'seas', 'species', 'stocks', 'sustainable', 'temperature', 'unregulated', 'warming', 'water']
sdg15 = ['afforestation', 'agriculture', 'alien', 'animals', 'arable', 'bees', 'biodiversity', 'conservation', 'deforestation', 'degradation', 'desertification', 'drought', 'drylands', 'ecosystem', 'ecosystems', 'extinct', 'extinction', 'fauna', 'flora', 'forest', 'forests', 'genetic', 'illegal', 'illicit', 'indigenous', 'invasive', 'land', 'manage', 'managed', 'management', 'microorganisms', 'permaculture', 'plan', 'plants', 'poaching', 'populations', 'poverty', 'products', 'protected', 'reforestation', 'resources', 'restoration', 'soil', 'species', 'strategic', 'sustainability', 'terrestrial', 'threatened', 'trafficking', 'tree', 'wetlands', 'wildlife']
sdg16 = ['abuse', 'access', 'accountability', 'accountable', 'against', 'arbitrary', 'arms', 'assets', 'birth', 'bribery', 'children', 'combat', 'conflict', 'conflicts', 'corruption', 'crime', 'decision_making', 'detainees', 'detention', 'disappearance', 'discrimination', 'displaced', 'education', 'enforced', 'equal', 'equity', 'evasion', 'exploitation', 'financial', 'freedom', 'geography', 'governance', 'hate', 'human', 'identity', 'illegal', 'illicit', 'inclusion', 'inclusive', 'institutions', 'internally', 'judiciary', 'justice', 'law', 'legal', 'life', 'national', 'nonviolence', 'organized', 'paris', 'peace', 'peaceful', 'physical', 'police', 'policy', 'poverty', 'prevent', 'principles', 'psychological', 'public', 'quality', 'rates', 'registration', 'representative', 'resolution', 'rights', 'rule', 'security', 'seizures', 'sexual', 'societies', 'societies', 'society', 'stolen', 'tax', 'terrorism', 'theft', 'threats', 'torture', 'trafficking', 'transparency', 'sentenced', 'unstable', 'victims', 'violence', 'weapon', 'women']
sdg17 = ['agenda', 'agreements', 'aid', 'assistance', 'average', 'building', 'capacity', 'census', 'civil', 'communication', 'cooperation', 'countries', 'data', 'debt', 'developing', 'development', 'disaggregated', 'doha', 'entrepreneurs', 'entrepreneurship', 'environmentally', 'eradication', 'foreign', 'fostering', 'fundamental', 'global', 'housing', 'innovation', 'international', 'investments', 'knowledge', 'stakeholder', 'official', 'organization', 'partnership', 'partnerships', 'population', 'poverty', 'principles', 'public', 'private', 'science', 'sharing', 'society', 'sound', 'stability', 'statistics', 'support', 'sustainability', 'sustainable', 'tariff', 'technologies', 'technology', 'trade', 'transfer', 'weighted', 'women', 'world']
sdg_list = [sdg1, sdg2, sdg3, sdg4, sdg5, sdg6, sdg7, sdg8, sdg9, sdg10, sdg11, sdg12, sdg13, sdg14, sdg15, sdg16, sdg17]

# KEYWORD matching (word_count/doc_len)

In [13]:
#sdg bag of words
sdg_bow = [id2word.doc2bow(text) for text in sdg_list]

In [14]:
# vectorized the sdg bag of words into a vector
sdg_vectorized = np.zeros((len(sdg_list), len(id2word)))
for i, sdg_bow_i in enumerate(sdg_bow):
    for id, count in sdg_bow_i:
        sdg_vectorized[i][id] = count

In [15]:
#vectorized each document (bag of words) into a vector
corpus_vectorized = np.zeros((len(corpus), len(id2word)))
for i, corpus_i in enumerate(corpus):
    for id, count in corpus_i:
        corpus_vectorized[i][id] = count

In [16]:
#total sdg words matched in a document
sdg_word_match = np.dot(corpus_vectorized,sdg_vectorized.transpose())

#total number of words in each document
total_words = corpus_vectorized.sum(axis=1)

sdg_contr = list()
#filename_list = list()

# probability estimation
for i, corpus_i in enumerate(total_words):
    sdg_contr.append(sdg_word_match[i]/corpus_i)
    #filename_list.append(df1['Project Name'][i])
    
# round off to 4 decimal points
#sdg_contr = np.around(sdg_contr, 4)

#sdg_curricula = sdg_contr[:166,:]
#sdg_framework = sdg_contr[166:,:]
#file_curricula = filename_list[:166]
#file_framework = filename_list[166:]

# create a dataframe for sdg probabilities of each curricula with filename
project_sdg_df = pd.DataFrame(data=sdg_contr)
#project_sdg_df['FILE NAME'] = filename_list

# create a dataframe for sdg probabilities of each curricula with filename
#framework_sdg_df = pd.DataFrame(data=sdg_framework)
#framework_sdg_df['FILE NAME'] = file_framework

#rename the columns
columns = ['sdg1', 'sdg2', 'sdg3', 'sdg4', 'sdg5', 'sdg6', 'sdg7', 'sdg8', 'sdg9', 'sdg10', 'sdg11', 'sdg12', 'sdg13', 'sdg14', 'sdg15', 'sdg16', 'sdg17']
project_sdg_df.columns = columns
#curricula_sdg_df.columns = columns

#project_sdg_df.to_excel("KW1.xlsx", engine='xlsxwriter')
#curricula_sdg_df.to_excel("curricula_sdg.xlsx", engine='xlsxwriter')

In [17]:
project_sdg_df

,sdg1,sdg2,sdg3,sdg4,sdg5,sdg6,sdg7,sdg8,sdg9,sdg10,sdg11,sdg12,sdg13,sdg14,sdg15,sdg16,sdg17
0,0.024786,0.025739,0.052431,0.029552,0.011439,0.028599,0.015253,0.026692,0.034318,0.039085,0.134414,0.034318,0.032412,0.018112,0.028599,0.023832,0.035272
1,0.040441,0.013480,0.045343,0.037990,0.053922,0.003676,0.001225,0.057598,0.000000,0.165441,0.017157,0.018382,0.007353,0.007353,0.023284,0.060049,0.045343
2,0.022518,0.021494,0.042989,0.011259,0.002047,0.014330,0.019447,0.021494,0.007165,0.022518,0.136131,0.047083,0.079836,0.031730,0.036847,0.014330,0.023541
3,0.003268,0.001089,0.165577,0.002179,0.013072,0.008715,0.002179,0.001089,0.003268,0.004357,0.016340,0.002179,0.017429,0.001089,0.003268,0.023965,0.018519
4,0.014403,0.106996,0.005144,0.009259,0.006173,0.010288,0.014403,0.060700,0.017490,0.016461,0.045267,0.095679,0.009259,0.007202,0.024691,0.011317,0.008230
5,0.004292,0.027897,0.019313,0.025751,0.089056,0.008584,0.012876,0.258584,0.022532,0.036481,0.074034,0.037554,0.005365,0.007511,0.006438,0.027897,0.011803
6,0.000883,0.000000,0.116505,0.003530,0.037070,0.000000,0.000883,0.011474,0.006178,0.004413,0.007944,0.000000,0.003530,0.000883,0.002648,0.157105,0.011474
7,0.014706,0.001961,0.206863,0.010784,0.223529,0.000980,0.000980,0.010784,0.000980,0.026471,0.017647,0.000980,0.001961,0.023529,0.002941,0.258824,0.004902
8,0.000000,0.024570,0.017199,0.015971,0.031941,0.011057,0.028256,0.085995,0.128993,0.022113,0.030713,0.060197,0.007371,0.003686,0.000000,0.009828,0.039312
9,0.005030,0.008048,0.012072,0.010060,0.001006,0.022133,0.004024,0.009054,0.015091,0.006036,0.024145,0.022133,0.025151,0.130785,0.101610,0.016097,0.013078


In [18]:
from sklearn import preprocessing
min_max_scaler = preprocessing.MinMaxScaler()
norm_sdg_df = pd.DataFrame(min_max_scaler.fit_transform(project_sdg_df.T).T)
norm_sdg_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,0.108527,0.116279,0.333333,0.147287,0.000000,0.139535,0.031008,0.124031,0.186047,0.224806,1.000000,0.186047,0.170543,0.054264,0.139535,0.100775,0.193798
1,0.244444,0.081481,0.274074,0.229630,0.325926,0.022222,0.007407,0.348148,0.000000,1.000000,0.103704,0.111111,0.044444,0.044444,0.140741,0.362963,0.274074
2,0.152672,0.145038,0.305344,0.068702,0.000000,0.091603,0.129771,0.145038,0.038168,0.152672,1.000000,0.335878,0.580153,0.221374,0.259542,0.091603,0.160305
3,0.013245,0.000000,1.000000,0.006623,0.072848,0.046358,0.006623,0.000000,0.013245,0.019868,0.092715,0.006623,0.099338,0.000000,0.013245,0.139073,0.105960
4,0.090909,1.000000,0.000000,0.040404,0.010101,0.050505,0.090909,0.545455,0.121212,0.111111,0.393939,0.888889,0.040404,0.020202,0.191919,0.060606,0.030303
5,0.000000,0.092827,0.059072,0.084388,0.333333,0.016878,0.033755,1.000000,0.071730,0.126582,0.274262,0.130802,0.004219,0.012658,0.008439,0.092827,0.029536
6,0.005618,0.000000,0.741573,0.022472,0.235955,0.000000,0.005618,0.073034,0.039326,0.028090,0.050562,0.000000,0.022472,0.005618,0.016854,1.000000,0.073034
7,0.053232,0.003802,0.798479,0.038023,0.863118,0.000000,0.000000,0.038023,0.000000,0.098859,0.064639,0.000000,0.003802,0.087452,0.007605,1.000000,0.015209
8,0.000000,0.190476,0.133333,0.123810,0.247619,0.085714,0.219048,0.666667,1.000000,0.171429,0.238095,0.466667,0.057143,0.028571,0.000000,0.076190,0.304762
9,0.031008,0.054264,0.085271,0.069767,0.000000,0.162791,0.023256,0.062016,0.108527,0.038760,0.178295,0.162791,0.186047,1.000000,0.775194,0.116279,0.093023


In [19]:
#import numpy
threshold = np.argsort(project_sdg_df)['sdg17']
df2.drop('filename', axis=1, inplace=True)

In [37]:
doc_num = 17
i = 0
match = 0
cutoff = 0.5

for i in range(doc_num):
    # use for top-3
    threshold = np.argsort(project_sdg_df)['sdg17']
    th1 = list(project_sdg_df.iloc[i] >= project_sdg_df.iloc[i][columns[threshold[i]]])
    
    #use for score > 0.7
    #th1 = list(norm_sdg_df.iloc[i] >= cutoff)
    th2 = list(df2.iloc[i])
    found = np.dot(th1, th2)
    if found > 0:
        match += 1
    else:
        print( th1, th2)
print("Accuracy Score: %.2f" % (match/doc_num*100) +"%")

[False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False] [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]
[False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False] [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False] [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
[False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False] [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Accuracy Score: 76.47%


# KEYWORD matching (word_match/sdg_len)

In [56]:
#sdg bag of words
sdg_bow = [id2word.doc2bow(text) for text in sdg_list]

In [57]:
# vectorized the sdg bag of words into a vector
sdg_vectorized = np.zeros((len(sdg_list), len(id2word)))
for i, sdg_bow_i in enumerate(sdg_bow):
    for id, count in sdg_bow_i:
        sdg_vectorized[i][id] = 1

In [58]:
#vectorized each document (bag of words) into a vector
corpus_vectorized = np.zeros((len(corpus), len(id2word)))
for i, corpus_i in enumerate(corpus):
    for id, count in corpus_i:
        corpus_vectorized[i][id] = 1

In [59]:
#total sdg words matched in a document
sdg_word_match = np.dot(corpus_vectorized,sdg_vectorized.transpose())

#total number of words in each document
#total_words = corpus_vectorized.sum(axis=1)
total_words = sdg_vectorized.sum(axis=1)

sdg_contr = list()
#filename_list = list()

# probability estimation
for i, corpus_i in enumerate(total_words):
    sdg_contr.append(sdg_word_match.T[i]/corpus_i)
    #filename_list.append(df1['Project Name'][i])
    
# round off to 4 decimal points
#sdg_contr = np.around(sdg_contr, 4)

#sdg_curricula = sdg_contr[:166,:]
#sdg_framework = sdg_contr[166:,:]
#file_curricula = filename_list[:166]
#file_framework = filename_list[166:]

# create a dataframe for sdg probabilities of each curricula with filename
project_sdg_df = pd.DataFrame(data=sdg_contr)
project_sdg_df = project_sdg_df.transpose()
#project_sdg_df['FILE NAME'] = filename_list

# create a dataframe for sdg probabilities of each curricula with filename
#framework_sdg_df = pd.DataFrame(data=sdg_framework)
#framework_sdg_df['FILE NAME'] = file_framework

#rename the columns
columns = ['sdg1', 'sdg2', 'sdg3', 'sdg4', 'sdg5', 'sdg6', 'sdg7', 'sdg8', 'sdg9', 'sdg10', 'sdg11', 'sdg12', 'sdg13', 'sdg14', 'sdg15', 'sdg16', 'sdg17']
project_sdg_df.columns = columns
#curricula_sdg_df.columns = columns

#project_sdg_df.to_excel("KW2.xlsx", engine='xlsxwriter')
#curricula_sdg_df.to_excel("curricula_sdg.xlsx", engine='xlsxwriter')

In [60]:
project_sdg_df

,sdg1,sdg2,sdg3,sdg4,sdg5,sdg6,sdg7,sdg8,sdg9,sdg10,sdg11,sdg12,sdg13,sdg14,sdg15,sdg16,sdg17
0,0.400000,0.368421,0.478261,0.296296,0.20,0.444444,0.391304,0.333333,0.461538,0.379310,0.800000,0.322581,0.65,0.50,0.40625,0.195652,0.428571
1,0.466667,0.105263,0.239130,0.370370,0.52,0.074074,0.043478,0.333333,0.000000,0.689655,0.171429,0.129032,0.10,0.05,0.12500,0.304348,0.428571
2,0.266667,0.315789,0.391304,0.296296,0.08,0.296296,0.304348,0.250000,0.192308,0.275862,0.657143,0.193548,0.70,0.45,0.34375,0.152174,0.357143
3,0.133333,0.052632,0.717391,0.037037,0.20,0.185185,0.043478,0.027778,0.076923,0.103448,0.200000,0.064516,0.20,0.05,0.06250,0.086957,0.142857
4,0.333333,0.894737,0.086957,0.222222,0.12,0.148148,0.173913,0.333333,0.230769,0.241379,0.228571,0.193548,0.15,0.10,0.28125,0.130435,0.178571
5,0.200000,0.263158,0.173913,0.370370,0.28,0.185185,0.304348,0.888889,0.423077,0.379310,0.257143,0.322581,0.10,0.10,0.09375,0.173913,0.178571
6,0.066667,0.000000,0.195652,0.074074,0.24,0.000000,0.043478,0.166667,0.115385,0.068966,0.085714,0.000000,0.10,0.05,0.06250,0.695652,0.214286
7,0.200000,0.052632,0.195652,0.185185,0.60,0.037037,0.043478,0.111111,0.038462,0.310345,0.171429,0.032258,0.10,0.05,0.06250,0.282609,0.142857
8,0.000000,0.263158,0.108696,0.185185,0.24,0.074074,0.391304,0.611111,0.769231,0.241379,0.171429,0.322581,0.15,0.05,0.00000,0.108696,0.321429
9,0.200000,0.210526,0.152174,0.148148,0.04,0.259259,0.130435,0.166667,0.230769,0.103448,0.342857,0.354839,0.40,0.90,0.43750,0.217391,0.250000


In [61]:
from sklearn import preprocessing
min_max_scaler = preprocessing.MinMaxScaler()
norm_sdg_df = pd.DataFrame(min_max_scaler.fit_transform(project_sdg_df.T).T)
norm_sdg_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,0.338129,0.285877,0.467626,0.166533,0.007194,0.411671,0.323741,0.227818,0.439956,0.303895,1.000000,0.210026,0.751799,0.503597,0.348471,0.000000,0.385406
1,0.676667,0.152632,0.346739,0.537037,0.754000,0.107407,0.063043,0.483333,0.000000,1.000000,0.248571,0.187097,0.145000,0.072500,0.181250,0.441304,0.621429
2,0.301075,0.380306,0.502104,0.348865,0.000000,0.348865,0.361851,0.274194,0.181141,0.315907,0.930876,0.183143,1.000000,0.596774,0.425403,0.116410,0.447005
3,0.153065,0.036040,1.000000,0.013427,0.249737,0.228255,0.022767,0.000000,0.071265,0.109729,0.249737,0.053274,0.249737,0.032224,0.050350,0.085814,0.166875
4,0.305005,1.000000,0.000000,0.167454,0.040907,0.075753,0.107649,0.305005,0.178034,0.191169,0.175314,0.131957,0.078045,0.016147,0.240528,0.053824,0.113416
5,0.133624,0.213054,0.100816,0.347889,0.234236,0.114993,0.264857,1.000000,0.414175,0.359133,0.205490,0.287787,0.007860,0.007860,0.000000,0.100816,0.106675
6,0.095833,0.000000,0.281250,0.106481,0.345000,0.000000,0.062500,0.239583,0.165865,0.099138,0.123214,0.000000,0.143750,0.071875,0.089844,1.000000,0.308036
7,0.295455,0.035885,0.287796,0.269360,1.000000,0.008418,0.019763,0.138889,0.010927,0.489812,0.245130,0.000000,0.119318,0.031250,0.053267,0.440958,0.194805
8,0.000000,0.342105,0.141304,0.240741,0.312000,0.096296,0.508696,0.794444,1.000000,0.313793,0.222857,0.419355,0.195000,0.065000,0.000000,0.141304,0.417857
9,0.186047,0.198286,0.130435,0.125754,0.000000,0.254953,0.105157,0.147287,0.221825,0.073777,0.352159,0.366092,0.418605,1.000000,0.462209,0.206269,0.244186


In [62]:
doc_num = 17
i = 0
match = 0

for i in range(doc_num):
    # use for top-3
    threshold = np.argsort(project_sdg_df)['sdg17']
    th1 = list(project_sdg_df.iloc[i] >= project_sdg_df.iloc[i][columns[threshold[i]]])
    
    #use for score > 0.7
    #th1 = list(norm_sdg_df.iloc[i] >= cutoff)
    
    th2 = list(df2.iloc[i])
    found = np.dot(th1, th2)
    if found > 0:
        match += 1
print("Accuracy Score: %.2f"% (match/doc_num*100)+"%")

Accuracy Score: 100.00%


In [33]:
np.argsort(project_sdg_df)['sdg17']

0     10
1      9
2     12
3      2
4      1
5      7
6     15
7      4
8      8
9     13
10    16
11     5
12     0
13    14
14     6
15     3
16    11
Name: sdg17, dtype: int64

# KEYWORD matching (word_count/sdg_len)

In [78]:
#sdg bag of words
sdg_bow = [id2word.doc2bow(text) for text in sdg_list]

In [79]:
# vectorized the sdg bag of words into a vector
sdg_vectorized = np.zeros((len(sdg_list), len(id2word)))
for i, sdg_bow_i in enumerate(sdg_bow):
    for id, count in sdg_bow_i:
        sdg_vectorized[i][id] = count

In [80]:
#vectorized each document (bag of words) into a vector
corpus_vectorized = np.zeros((len(corpus), len(id2word)))
for i, corpus_i in enumerate(corpus):
    for id, count in corpus_i:
        corpus_vectorized[i][id] = count

In [81]:
#total sdg words matched in a document
sdg_word_match = np.dot(corpus_vectorized,sdg_vectorized.transpose())

#total number of words in each document
#total_words = corpus_vectorized.sum(axis=1)
total_words = sdg_vectorized.sum(axis=1)

sdg_contr = list()
#filename_list = list()

# probability estimation
for i, corpus_i in enumerate(total_words):
    sdg_contr.append(sdg_word_match.T[i]/corpus_i)
    #filename_list.append(df1['Project Name'][i])
    
# round off to 4 decimal points
#sdg_contr = np.around(sdg_contr, 4)

#sdg_curricula = sdg_contr[:166,:]
#sdg_framework = sdg_contr[166:,:]
#file_curricula = filename_list[:166]
#file_framework = filename_list[166:]

# create a dataframe for sdg probabilities of each curricula with filename
project_sdg_df = pd.DataFrame(data=sdg_contr)
project_sdg_df = project_sdg_df.transpose()
#project_sdg_df['FILE NAME'] = filename_list

# create a dataframe for sdg probabilities of each curricula with filename
#framework_sdg_df = pd.DataFrame(data=sdg_framework)
#framework_sdg_df['FILE NAME'] = file_framework

#rename the columns
columns = ['sdg1', 'sdg2', 'sdg3', 'sdg4', 'sdg5', 'sdg6', 'sdg7', 'sdg8', 'sdg9', 'sdg10', 'sdg11', 'sdg12', 'sdg13', 'sdg14', 'sdg15', 'sdg16', 'sdg17']
project_sdg_df.columns = columns
#curricula_sdg_df.columns = columns

#project_sdg_df.to_excel("KW3.xlsx", engine='xlsxwriter')
#curricula_sdg_df.to_excel("curricula_sdg.xlsx", engine='xlsxwriter')

In [82]:
project_sdg_df

,sdg1,sdg2,sdg3,sdg4,sdg5,sdg6,sdg7,sdg8,sdg9,sdg10,sdg11,sdg12,sdg13,sdg14,sdg15,sdg16,sdg17
0,1.733333,1.421053,1.195652,1.148148,0.48,1.111111,0.695652,0.777778,1.384615,1.413793,4.028571,1.161290,1.70,0.95,0.93750,0.543478,1.321429
1,2.200000,0.578947,0.804348,1.148148,1.76,0.111111,0.043478,1.305556,0.000000,4.655172,0.400000,0.483871,0.30,0.30,0.59375,1.065217,1.321429
2,1.466667,1.105263,0.913043,0.407407,0.08,0.518519,0.826087,0.583333,0.269231,0.758621,3.800000,1.483871,3.90,1.55,1.12500,0.304348,0.821429
3,0.200000,0.052632,3.304348,0.074074,0.48,0.296296,0.086957,0.027778,0.115385,0.137931,0.428571,0.064516,0.80,0.05,0.09375,0.478261,0.607143
4,0.933333,5.473684,0.108696,0.333333,0.24,0.370370,0.608696,1.638889,0.653846,0.551724,1.257143,3.000000,0.45,0.35,0.75000,0.239130,0.285714
5,0.266667,1.368421,0.391304,0.888889,3.32,0.296296,0.521739,6.694444,0.807692,1.172414,1.971429,1.129032,0.25,0.35,0.18750,0.565217,0.392857
6,0.066667,0.000000,2.869565,0.148148,1.68,0.000000,0.043478,0.361111,0.269231,0.172414,0.257143,0.000000,0.20,0.05,0.09375,3.869565,0.464286
7,1.000000,0.105263,4.586957,0.407407,9.12,0.037037,0.043478,0.305556,0.038462,0.931034,0.514286,0.032258,0.10,1.20,0.09375,5.739130,0.178571
8,0.000000,1.052632,0.304348,0.481481,1.04,0.333333,1.000000,1.944444,4.038462,0.620690,0.714286,1.580645,0.30,0.15,0.00000,0.173913,1.142857
9,0.333333,0.421053,0.260870,0.370370,0.04,0.814815,0.173913,0.250000,0.576923,0.206897,0.685714,0.709677,1.25,6.50,3.15625,0.347826,0.464286


In [83]:
from sklearn import preprocessing
min_max_scaler = preprocessing.MinMaxScaler()
norm_sdg_df = pd.DataFrame(min_max_scaler.fit_transform(project_sdg_df.T).T)
norm_sdg_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,0.353194,0.265192,0.201673,0.188287,0.000000,0.177849,0.060772,0.083915,0.254924,0.263146,1.000000,0.191990,0.343800,0.132448,0.128925,0.017888,0.237118
1,0.472593,0.124366,0.172786,0.246639,0.378074,0.023868,0.009340,0.280453,0.000000,1.000000,0.085926,0.103943,0.064444,0.064444,0.127546,0.228824,0.283862
2,0.363002,0.268393,0.218074,0.085709,0.000000,0.114795,0.195311,0.131763,0.049537,0.177649,0.973822,0.367505,1.000000,0.384817,0.273560,0.058730,0.194091
3,0.052562,0.007585,1.000000,0.014129,0.138017,0.081951,0.018061,0.000000,0.026737,0.033618,0.122321,0.011212,0.235680,0.006782,0.020135,0.137486,0.176821
4,0.153707,1.000000,0.000000,0.041871,0.024474,0.048775,0.093197,0.285218,0.101613,0.082578,0.214063,0.538921,0.063617,0.044978,0.119535,0.024312,0.032995
5,0.012166,0.181486,0.031321,0.107791,0.481409,0.016720,0.051367,1.000000,0.095312,0.151363,0.274158,0.144697,0.009605,0.024973,0.000000,0.058048,0.031560
6,0.017228,0.000000,0.741573,0.038285,0.434157,0.000000,0.011236,0.093321,0.069576,0.044556,0.066453,0.000000,0.051685,0.012921,0.024228,1.000000,0.119984
7,0.106489,0.008033,0.501191,0.041281,1.000000,0.000526,0.001235,0.030073,0.000683,0.098900,0.053042,0.000000,0.007454,0.128496,0.006766,0.627975,0.016100
8,0.000000,0.260652,0.075362,0.119224,0.257524,0.082540,0.247619,0.481481,1.000000,0.153695,0.176871,0.391398,0.074286,0.037143,0.000000,0.043064,0.282993
9,0.045408,0.058986,0.034190,0.051141,0.000000,0.119940,0.020730,0.032508,0.083115,0.025835,0.099956,0.103665,0.187307,1.000000,0.482392,0.047651,0.065679


In [84]:
doc_num = 17
i = 0
match = 0
cutoff = 0.8

for i in range(doc_num):
    # use for top-1
    #threshold = np.argsort(project_sdg_df)['sdg17']
    #th1 = list(project_sdg_df.iloc[i] >= project_sdg_df.iloc[i][columns[threshold[i]]])
    
    #use for score > 0.7
    th1 = list(norm_sdg_df.iloc[i] >= cutoff)
    
    th2 = list(df2.iloc[i])
    found = np.dot(th1, th2)
    if found > 0:
        match += 1
print("Accuracy Score: %.2f" % (match/doc_num*100) +"%")

Accuracy Score: 100.00%


# KEYWORD matching (word_count/sdg_len*doc_len)

In [85]:
#sdg bag of words
sdg_bow = [id2word.doc2bow(text) for text in sdg_list]

In [86]:
# vectorized the sdg bag of words into a vector
sdg_vectorized = np.zeros((len(sdg_list), len(id2word)))
for i, sdg_bow_i in enumerate(sdg_bow):
    for id, count in sdg_bow_i:
        sdg_vectorized[i][id] = count

In [87]:
#vectorized each document (bag of words) into a vector
corpus_vectorized = np.zeros((len(corpus), len(id2word)))
for i, corpus_i in enumerate(corpus):
    for id, count in corpus_i:
        corpus_vectorized[i][id] = count

In [88]:
#total sdg words matched in a document
sdg_word_match = np.dot(corpus_vectorized,sdg_vectorized.transpose())

#total number of words in each document
doc_words = corpus_vectorized.sum(axis=1)
sdg_words = sdg_vectorized.sum(axis=1)

sdg_contr = list()
#filename_list = list()

# probability estimation
for i, corpus_i in enumerate(sdg_words):
    sdg_contr.append(sdg_word_match.T[i]/corpus_i)

# create a dataframe for sdg probabilities of each curricula with filename
project_sdg_df = pd.DataFrame(data=sdg_contr)
project_sdg_df = project_sdg_df.transpose()
project_sdg_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,1.733333,1.421053,1.195652,1.148148,0.48,1.111111,0.695652,0.777778,1.384615,1.413793,4.028571,1.161290,1.70,0.95,0.93750,0.543478,1.321429
1,2.200000,0.578947,0.804348,1.148148,1.76,0.111111,0.043478,1.305556,0.000000,4.655172,0.400000,0.483871,0.30,0.30,0.59375,1.065217,1.321429
2,1.466667,1.105263,0.913043,0.407407,0.08,0.518519,0.826087,0.583333,0.269231,0.758621,3.800000,1.483871,3.90,1.55,1.12500,0.304348,0.821429
3,0.200000,0.052632,3.304348,0.074074,0.48,0.296296,0.086957,0.027778,0.115385,0.137931,0.428571,0.064516,0.80,0.05,0.09375,0.478261,0.607143
4,0.933333,5.473684,0.108696,0.333333,0.24,0.370370,0.608696,1.638889,0.653846,0.551724,1.257143,3.000000,0.45,0.35,0.75000,0.239130,0.285714
5,0.266667,1.368421,0.391304,0.888889,3.32,0.296296,0.521739,6.694444,0.807692,1.172414,1.971429,1.129032,0.25,0.35,0.18750,0.565217,0.392857
6,0.066667,0.000000,2.869565,0.148148,1.68,0.000000,0.043478,0.361111,0.269231,0.172414,0.257143,0.000000,0.20,0.05,0.09375,3.869565,0.464286
7,1.000000,0.105263,4.586957,0.407407,9.12,0.037037,0.043478,0.305556,0.038462,0.931034,0.514286,0.032258,0.10,1.20,0.09375,5.739130,0.178571
8,0.000000,1.052632,0.304348,0.481481,1.04,0.333333,1.000000,1.944444,4.038462,0.620690,0.714286,1.580645,0.30,0.15,0.00000,0.173913,1.142857
9,0.333333,0.421053,0.260870,0.370370,0.04,0.814815,0.173913,0.250000,0.576923,0.206897,0.685714,0.709677,1.25,6.50,3.15625,0.347826,0.464286


In [89]:
sdg_contr = list()
for i, corpus_i in enumerate(doc_words):
    sdg_contr.append(list(project_sdg_df.iloc[i]/corpus_i))
    
project_sdg_df = pd.DataFrame(data=sdg_contr)

#rename the columns
columns = ['sdg1', 'sdg2', 'sdg3', 'sdg4', 'sdg5', 'sdg6', 'sdg7', 'sdg8', 'sdg9', 'sdg10', 'sdg11', 'sdg12', 'sdg13', 'sdg14', 'sdg15', 'sdg16', 'sdg17']
project_sdg_df.columns = columns
project_sdg_df


#project_sdg_df.to_excel("KW4.xlsx", engine='xlsxwriter')

,sdg1,sdg2,sdg3,sdg4,sdg5,sdg6,sdg7,sdg8,sdg9,sdg10,sdg11,sdg12,sdg13,sdg14,sdg15,sdg16,sdg17
0,0.001652,0.001355,0.001140,0.001095,0.000458,0.001059,0.000663,0.000741,0.001320,0.001348,0.003840,0.001107,0.001621,0.000906,0.000894,0.000518,0.001260
1,0.002696,0.000709,0.000986,0.001407,0.002157,0.000136,0.000053,0.001600,0.000000,0.005705,0.000490,0.000593,0.000368,0.000368,0.000728,0.001305,0.001619
2,0.001501,0.001131,0.000935,0.000417,0.000082,0.000531,0.000846,0.000597,0.000276,0.000776,0.003889,0.001519,0.003992,0.001586,0.001151,0.000312,0.000841
3,0.000218,0.000057,0.003600,0.000081,0.000523,0.000323,0.000095,0.000030,0.000126,0.000150,0.000467,0.000070,0.000871,0.000054,0.000102,0.000521,0.000661
4,0.000960,0.005631,0.000112,0.000343,0.000247,0.000381,0.000626,0.001686,0.000673,0.000568,0.001293,0.003086,0.000463,0.000360,0.000772,0.000246,0.000294
5,0.000286,0.001468,0.000420,0.000954,0.003562,0.000318,0.000560,0.007183,0.000867,0.001258,0.002115,0.001211,0.000268,0.000376,0.000201,0.000606,0.000422
6,0.000059,0.000000,0.002533,0.000131,0.001483,0.000000,0.000038,0.000319,0.000238,0.000152,0.000227,0.000000,0.000177,0.000044,0.000083,0.003415,0.000410
7,0.000980,0.000103,0.004497,0.000399,0.008941,0.000036,0.000043,0.000300,0.000038,0.000913,0.000504,0.000032,0.000098,0.001176,0.000092,0.005627,0.000175
8,0.000000,0.001293,0.000374,0.000592,0.001278,0.000410,0.001229,0.002389,0.004961,0.000763,0.000878,0.001942,0.000369,0.000184,0.000000,0.000214,0.001404
9,0.000335,0.000424,0.000262,0.000373,0.000040,0.000820,0.000175,0.000252,0.000580,0.000208,0.000690,0.000714,0.001258,0.006539,0.003175,0.000350,0.000467


In [90]:
from sklearn import preprocessing
min_max_scaler = preprocessing.MinMaxScaler()
norm_sdg_df = pd.DataFrame(min_max_scaler.fit_transform(project_sdg_df.T).T)
norm_sdg_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,0.353194,0.265192,0.201673,0.188287,0.000000,0.177849,0.060772,0.083915,0.254924,0.263146,1.000000,0.191990,0.343800,0.132448,0.128925,0.017888,0.237118
1,0.472593,0.124366,0.172786,0.246639,0.378074,0.023868,0.009340,0.280453,0.000000,1.000000,0.085926,0.103943,0.064444,0.064444,0.127546,0.228824,0.283862
2,0.363002,0.268393,0.218074,0.085709,0.000000,0.114795,0.195311,0.131763,0.049537,0.177649,0.973822,0.367505,1.000000,0.384817,0.273560,0.058730,0.194091
3,0.052562,0.007585,1.000000,0.014129,0.138017,0.081951,0.018061,0.000000,0.026737,0.033618,0.122321,0.011212,0.235680,0.006782,0.020135,0.137486,0.176821
4,0.153707,1.000000,0.000000,0.041871,0.024474,0.048775,0.093197,0.285218,0.101613,0.082578,0.214063,0.538921,0.063617,0.044978,0.119535,0.024312,0.032995
5,0.012166,0.181486,0.031321,0.107791,0.481409,0.016720,0.051367,1.000000,0.095312,0.151363,0.274158,0.144697,0.009605,0.024973,0.000000,0.058048,0.031560
6,0.017228,0.000000,0.741573,0.038285,0.434157,0.000000,0.011236,0.093321,0.069576,0.044556,0.066453,0.000000,0.051685,0.012921,0.024228,1.000000,0.119984
7,0.106489,0.008033,0.501191,0.041281,1.000000,0.000526,0.001235,0.030073,0.000683,0.098900,0.053042,0.000000,0.007454,0.128496,0.006766,0.627975,0.016100
8,0.000000,0.260652,0.075362,0.119224,0.257524,0.082540,0.247619,0.481481,1.000000,0.153695,0.176871,0.391398,0.074286,0.037143,0.000000,0.043064,0.282993
9,0.045408,0.058986,0.034190,0.051141,0.000000,0.119940,0.020730,0.032508,0.083115,0.025835,0.099956,0.103665,0.187307,1.000000,0.482392,0.047651,0.065679


In [91]:
doc_num = 17
i = 0
match = 0

for i in range(doc_num):
    # use for top-1
    threshold = np.argsort(project_sdg_df)['sdg17']
    th1 = list(project_sdg_df.iloc[i] >= project_sdg_df.iloc[i][columns[threshold[i]]])
    
    #use for score > 0.7
    #th1 = list(norm_sdg_df.iloc[i] >= cutoff)
    th2 = list(df2.iloc[i])
    found = np.dot(th1, th2)
    if found > 0:
        match += 1
print("Accuracy Score: %.2f" % (match/doc_num*100) +"%")

Accuracy Score: 100.00%


# Similarity (1-gram) Threshold=0.6

In [92]:
import gensim.downloader as api
model = api.load("glove-wiki-gigaword-100")

In [93]:
#defining all 17 SDGs
sdg1 = ['africa', 'basic', 'class', 'countries', 'developing', 'disadvantaged', 'discrimination', 'distribution', 'economic', 'end', 'environment', 'equality', 'eradication', 'extreme', 'financial', 'inclusion', 'income', 'life', 'line', 'microfinance', 'poor', 'poverty', 'protection', 'quality', 'resources', 'services', 'social', 'sustainable', 'systems', 'third', 'vulnerable', 'wealth', 'world']
sdg2 = ['agriculture', 'diversity', 'food', 'genetic', 'genetics', 'growth', 'health', 'hunger', 'hungry', 'improved', 'infrastructure', 'innovations', 'legumes', 'life', 'maize', 'malnourished', 'malnutrition', 'needs', 'nourished', 'nutrition', 'nutritional', 'nutritious', 'people', 'poverty', 'produce', 'producers', 'production', 'productivity', 'quality', 'resilient', 'restrictions', 'rural', 'scale', 'seeds', 'small', 'stunted', 'stunting', 'sufficient', 'sustainable', 'trade', 'under', 'undernourished', 'wasting', 'world']
sdg3 = ['abuse', 'access', 'accidents', 'affordable', 'aids', 'air', 'alcohol', 'antenatal', 'antiretroviral', 'autonomy', 'biomedical', 'bodily', 'borne', 'care', 'child', 'clean', 'constrained', 'contamination', 'contraceptive', 'control', 'countries', 'coverage', 'death', 'deaths', 'density', 'dental', 'developing', 'disability', 'disease', 'diseases', 'drug', 'expectancy', 'family', 'health', 'healthcare', 'healthy', 'hepatitis', 'hiv', 'hygiene', 'improving', 'inclusion', 'increasing', 'indigenous', 'infected', 'international', 'life', 'lives', 'location', 'malaria', 'maternal', 'measles', 'medical', 'medicines', 'mental', 'mortality', 'narcotic', 'neonatal', 'organisation', 'planning', 'policy', 'polio', 'politics', 'pollution', 'premature', 'preventable', 'reducing', 'refugees', 'regulations', 'reproductive', 'resource', 'road', 'sanitation', 'services', 'sexual', 'soil', 'substance', 'support', 'therapy', 'tobacco', 'traffic', 'treatment', 'tuberculosis', 'universal', 'use', 'vaccines', 'violence', 'wash', 'water', 'well', 'wellbeing', 'worker', 'world']
sdg4 = ['access', 'basic', 'childhood', 'citizenship', 'cooperation', 'cultural', 'developing', 'development', 'disability', 'disparities', 'disparity', 'diversity', 'education', 'enrolment', 'equal', 'equality', 'equitable', 'equity', 'gender', 'global', 'inclusion', 'inclusive', 'innovation', 'international', 'learning', 'lifelong', 'literacy', 'numeracy', 'opportunities', 'preprimary', 'primary', 'qualified', 'refugees', 'rights', 'scholarships', 'school', 'secondary', 'sensitive', 'skills', 'sustainability', 'teacher', 'teachers', 'training', 'universal', 'vocational', 'vulnerable', 'women']
sdg5 = ['access', 'against', 'basic', 'coverage', 'dignity', 'disadvantaged', 'discrimination', 'employment', 'empower', 'empowerment', 'equal', 'equality', 'exploitation', 'female', 'feminism', 'forced', 'gender', 'genital', 'girls', 'governance', 'health', 'human', 'humanitarian', 'inclusion', 'living', 'marginalised', 'marriage', 'mutilation', 'opportunities', 'parity', 'pay', 'reproductive', 'rights', 'sexual', 'social', 'standards', 'trafficking', 'universal', 'violence', 'women', 'work', 'workplace']
sdg6 = ['access', 'accessible', 'affordable', 'aquifer', 'basins', 'cities', 'clean', 'contaminated', 'defecation', 'desalination', 'diarrhoeal', 'disasters', 'diseases', 'drinking', 'drought', 'dumping', 'ecosystem', 'ecosystems', 'efficiency', 'equitable', 'floods', 'harvesting', 'hydropower', 'hygiene', 'improving', 'inadequate', 'infrastructure', 'irrigation', 'lakes', 'latrines', 'management', 'pollution', 'protection', 'quality', 'recycled', 'resources', 'restoration', 'reuse', 'river', 'rivers', 'safe', 'sanitation', 'scarcity', 'sewerage', 'supply', 'sustainable', 'third', 'toilets', 'treatment', 'untreated', 'urban', 'waste', 'wastewater', 'water', 'water_related', 'water_use', 'withdrawals', 'world']
sdg7 = ['affordable', 'alternative', 'animal', 'battery', 'carbon', 'charcoal', 'clean', 'cleaner', 'climate', 'coal', 'economy', 'efficiency', 'electricity', 'emissions', 'energy', 'fossil', 'fossil_fuel', 'fuel', 'fuels', 'gas', 'goal', 'green', 'greenhouse', 'hydroelectric', 'infrastructure', 'modern', 'power', 'reliable', 'renewable', 'research', 'services', 'solar', 'sustainable', 'technology', 'turbine', 'vehicles', 'waste', 'wave', 'wind', 'wood']
sdg8 = ['aid', 'banking', 'child', 'consumption', 'creation', 'creativity', 'culture', 'development', 'domestic', 'economic', 'economy', 'efficiency', 'employment', 'enterprises', 'entrepreneurship', 'equal', 'eradication', 'finance', 'financial', 'forced', 'gdp', 'global', 'gross', 'growth', 'human', 'inclusive', 'innovation', 'insurance', 'job', 'jobs', 'labour', 'life', 'market', 'micro', 'migrant', 'migrants', 'modern', 'opportunities', 'oriented', 'pay', 'policies', 'policy', 'poverty', 'product', 'production', 'productive', 'productivity', 'public', 'quality', 'resource', 'rights', 'safe', 'secure', 'services', 'slavery', 'social', 'society', 'soldiers', 'stable', 'sustainable', 'tourism', 'trade', 'trafficking', 'unemployment', 'paid', 'women', 'work', 'workers', 'world', 'youth']
sdg9 = ['access', 'affordable', 'banks', 'capabilities', 'chains', 'clean', 'communication', 'cooperation', 'countries', 'credit', 'data', 'developing', 'development', 'diversification', 'economic', 'education', 'efficiency', 'electrical', 'energy', 'enterprises', 'environmentally', 'financial', 'ict', 'industrial', 'industrialisation', 'information', 'infrastructure', 'innovation', 'internet', 'irrigation', 'knowledge', 'markets', 'mobile', 'national', 'network', 'networks', 'phone', 'policy', 'power', 'public', 'quality', 'regional', 'research', 'resilient', 'resource', 'resources', 'roads', 'sanitation', 'scientific', 'security', 'service', 'services', 'society', 'sustainable', 'technological', 'technologies', 'technology', 'trade', 'transborder', 'transport', 'value', 'water']
sdg10 = ['affordable', 'age', 'ageism', 'aid', 'assistance', 'business', 'children', 'countries', 'culture', 'developing', 'development', 'disabilities', 'discrimination', 'discriminatory', 'economy', 'education', 'empower', 'equal', 'equality', 'equity', 'ethnicity', 'financial', 'foreign', 'gender', 'global', 'growth', 'health', 'homelessness', 'homophobia', 'housing', 'human', 'inclusion', 'income', 'indigenous', 'inequalities', 'inequality', 'investment', 'markets', 'migrant', 'migration', 'nations', 'opportunity', 'policy', 'population', 'poverty', 'protection', 'public', 'quality', 'race', 'racisim', 'reduce', 'religion', 'remittance', 'rights', 'rural', 'sex', 'sexism', 'social', 'society', 'states', 'trade', 'vulnerable', 'world']
sdg11 = ['adaptable', 'adaptation', 'affordable', 'air', 'buildings', 'building', 'change', 'cities', 'city', 'climate', 'communities', 'community', 'consumption', 'crowding', 'cultural', 'decentralisation', 'development', 'disaster', 'disasters', 'efficiency', 'fine', 'generation', 'green', 'growth', 'heritage', 'housing', 'human', 'impact', 'inadequate', 'informal', 'infrastructure', 'land', 'local', 'management', 'materials', 'matter', 'mitigation', 'natural', 'particulate', 'planning', 'pollution', 'population', 'public', 'quality', 'reduction', 'resilient', 'resource', 'risk', 'road', 'safe', 'safety', 'settlements', 'shanty', 'slums', 'smart', 'solid', 'spaces', 'strategy', 'suburban', 'sustainability', 'sustainable', 'systems', 'town', 'transport', 'urban', 'urbanisation', 'waste', 'water', 'water_related']
sdg12 = ['capitalism', 'cars', 'chain', 'chains', 'circular', 'commercial', 'consumer', 'consumerism', 'consumption', 'cycle', 'decarbonisation', 'development', 'distortions', 'ecological', 'economy', 'efficiency', 'efficient', 'energy', 'enterprises', 'food', 'fossil', 'fuel', 'future', 'gasses', 'generation', 'global', 'goods', 'greenhouse', 'harvest', 'industry', 'levels', 'losses', 'management', 'market', 'materialism', 'materials', 'monitoring', 'natural', 'obsolescence', 'overconsumption', 'pollution', 'practices', 'procurement', 'production', 'proof', 'public', 'recycle', 'recycling', 'reduce', 'reduction', 'renewable', 'resource', 'resources', 'responsible', 'retail', 'reuse', 'subsidies', 'supply', 'sustainable', 'tourism', 'vehicles', 'waste', 'wasteful', 'water']
sdg13 = ['action', 'adaptation', 'agreement', 'average', 'capture', 'carbon', 'change', 'changing', 'climate', 'co2', 'conversion', 'cop', 'dioxide', 'disasters', 'disease', 'economy', 'ecosystems', 'emissions', 'events', 'extreme', 'gas', 'gases', 'gender', 'global', 'greenhouse', 'hazards', 'ice', 'impact', 'infectious', 'management', 'mitigation', 'natural', 'ocean', 'paris', 'patterns', 'planning', 'policy', 'politics', 'pollution', 'refugees', 'related', 'renewable', 'resilience', 'rise', 'rising', 'sea', 'systems', 'temperature', 'warming', 'warning', 'weather']
sdg14 = ['acidification', 'areas', 'artisanal', 'biodiversity', 'bleaching', 'carbon', 'coastal', 'coastlines', 'conserve', 'coral', 'dioxide', 'ecosystem', 'ecosystems', 'fish', 'fisheries', 'fishers', 'fishing', 'global', 'grasses', 'habitats', 'illegal', 'kelp', 'law', 'management', 'marine', 'ocean', 'oceanography', 'oceans', 'overfishing', 'parks', 'policy', 'pollution', 'practices', 'productive', 'protected', 'reef', 'resources', 'sea', 'seas', 'species', 'stocks', 'sustainable', 'temperature', 'unregulated', 'warming', 'water']
sdg15 = ['afforestation', 'agriculture', 'alien', 'animals', 'arable', 'bees', 'biodiversity', 'conservation', 'deforestation', 'degradation', 'desertification', 'drought', 'drylands', 'ecosystem', 'ecosystems', 'extinct', 'extinction', 'fauna', 'flora', 'forest', 'forests', 'genetic', 'illegal', 'illicit', 'indigenous', 'invasive', 'land', 'manage', 'managed', 'management', 'microorganisms', 'permaculture', 'plan', 'plants', 'poaching', 'populations', 'poverty', 'products', 'protected', 'reforestation', 'resources', 'restoration', 'soil', 'species', 'strategic', 'sustainability', 'terrestrial', 'threatened', 'trafficking', 'tree', 'wetlands', 'wildlife']
sdg16 = ['abuse', 'access', 'accountability', 'accountable', 'against', 'arbitrary', 'arms', 'assets', 'birth', 'bribery', 'children', 'combat', 'conflict', 'conflicts', 'corruption', 'crime', 'decision_making', 'detainees', 'detention', 'disappearance', 'discrimination', 'displaced', 'education', 'enforced', 'equal', 'equity', 'evasion', 'exploitation', 'financial', 'freedom', 'geography', 'governance', 'hate', 'human', 'identity', 'illegal', 'illicit', 'inclusion', 'inclusive', 'institutions', 'internally', 'judiciary', 'justice', 'law', 'legal', 'life', 'national', 'nonviolence', 'organized', 'paris', 'peace', 'peaceful', 'physical', 'police', 'policy', 'poverty', 'prevent', 'principles', 'psychological', 'public', 'quality', 'rates', 'registration', 'representative', 'resolution', 'rights', 'rule', 'security', 'seizures', 'sexual', 'societies', 'societies', 'society', 'stolen', 'tax', 'terrorism', 'theft', 'threats', 'torture', 'trafficking', 'transparency', 'sentenced', 'unstable', 'victims', 'violence', 'weapon', 'women']
sdg17 = ['agenda', 'agreements', 'aid', 'assistance', 'average', 'building', 'capacity', 'census', 'civil', 'communication', 'cooperation', 'countries', 'data', 'debt', 'developing', 'development', 'disaggregated', 'doha', 'entrepreneurs', 'entrepreneurship', 'environmentally', 'eradication', 'foreign', 'fostering', 'fundamental', 'global', 'housing', 'innovation', 'international', 'investments', 'knowledge', 'stakeholder', 'official', 'organization', 'partnership', 'partnerships', 'population', 'poverty', 'principles', 'public', 'private', 'science', 'sharing', 'society', 'sound', 'stability', 'statistics', 'support', 'sustainability', 'sustainable', 'tariff', 'technologies', 'technology', 'trade', 'transfer', 'weighted', 'women', 'world']
sdg_list = [sdg1, sdg2, sdg3, sdg4, sdg5, sdg6, sdg7, sdg8, sdg9, sdg10, sdg11, sdg12, sdg13, sdg14, sdg15, sdg16, sdg17]

In [60]:
documents = data_lemmatized
sdg_l = sdg_list

score_vector1 = np.zeros((len(documents), len(sdg_l)))
score_vector2 = np.zeros((len(documents), len(sdg_l)))
score_vector3 = np.zeros((len(documents), len(sdg_l)))
score_vector4 = np.zeros((len(documents), len(sdg_l)))

for d, doc in tqdm(enumerate(documents)):
    d_len = len(documents[d])
    #print(d)
    for s, sdg in enumerate(sdg_l):
        s_len = len(sdg_l[s])
        score = 0
        for keyword in sdg:
            for word in doc:
                try:
                    sim = model.similarity(word, keyword)
                    if (sim >= 0.6):
                        score += sim 
                except:
                    pass
        score_vector1[d][s] = score
        score_vector2[d][s] = score / d_len
        score_vector3[d][s] = score / s_len
        score_vector4[d][s] = score / (d_len*s_len)

17it [15:14, 53.79s/it]


In [61]:
import numpy

project_sdg_df1 = pd.DataFrame(data=score_vector1)
project_sdg_df2 = pd.DataFrame(data=score_vector2)
project_sdg_df3 = pd.DataFrame(data=score_vector3)
project_sdg_df4 = pd.DataFrame(data=score_vector4)

columns = ['sdg1', 'sdg2', 'sdg3', 'sdg4', 'sdg5', 'sdg6', 'sdg7', 'sdg8', 'sdg9', 'sdg10', 'sdg11', 'sdg12', 'sdg13', 'sdg14', 'sdg15', 'sdg16', 'sdg17']

project_sdg_df1.columns = columns
project_sdg_df2.columns = columns
project_sdg_df3.columns = columns
project_sdg_df4.columns = columns

#threshold1 = numpy.argsort(project_sdg_df1)['sdg17']
#threshold2 = numpy.argsort(project_sdg_df2)['sdg17']
#threshold3 = numpy.argsort(project_sdg_df3)['sdg17']
#threshold4 = numpy.argsort(project_sdg_df4)['sdg17']

In [62]:
from sklearn import preprocessing
min_max_scaler = preprocessing.MinMaxScaler()
norm_sdg_df1 = pd.DataFrame(min_max_scaler.fit_transform(project_sdg_df1.T).T)
norm_sdg_df2 = pd.DataFrame(min_max_scaler.fit_transform(project_sdg_df2.T).T)
norm_sdg_df3 = pd.DataFrame(min_max_scaler.fit_transform(project_sdg_df3.T).T)
norm_sdg_df4 = pd.DataFrame(min_max_scaler.fit_transform(project_sdg_df4.T).T)

In [63]:
doc_num = 17
i = 0
match = 0

for i in range(doc_num):
    # use for top-1
    threshold = np.argsort(project_sdg_df1)['sdg17']
    th1 = list(project_sdg_df1.iloc[i] >= project_sdg_df1.iloc[i][columns[threshold[i]]])
    
    #use for score > 0.7
    #th1 = list(norm_sdg_df1.iloc[i] >= cutoff)
    th2 = list(df2.iloc[i])
    found = np.dot(th1, th2)
    if found > 0:
        match += 1
print("Accuracy Score (sim): %.2f"% (match/doc_num*100)+"%")

match=0
for i in range(doc_num):
    # use for top-1
    threshold = np.argsort(project_sdg_df2)['sdg17']
    th1 = list(project_sdg_df2.iloc[i] >= project_sdg_df2.iloc[i][columns[threshold[i]]])
    
    #use for score > 0.7
    #th1 = list(norm_sdg_df2.iloc[i] >= cutoff)
    th2 = list(df2.iloc[i])
    found = np.dot(th1, th2)
    if found > 0:
        match += 1
print("Accuracy Score (sim/doc_len): %.2f"% (match/doc_num*100)+"%")

match=0
for i in range(doc_num):
    # use for top-1
    threshold = np.argsort(project_sdg_df3)['sdg17']
    th1 = list(project_sdg_df3.iloc[i] >= project_sdg_df3.iloc[i][columns[threshold[i]]])
    
    #use for score > 0.7
    #th1 = list(norm_sdg_df3.iloc[i] >= cutoff)
    th2 = list(df2.iloc[i])
    found = np.dot(th1, th2)
    if found > 0:
        match += 1
print("Accuracy Score (sim/sdg_len): %.2f"% (match/doc_num*100)+"%")

match=0
for i in range(doc_num):
    # use for top-1
    threshold = np.argsort(project_sdg_df4)['sdg17']
    th1 = list(project_sdg_df4.iloc[i] >= project_sdg_df4.iloc[i][columns[threshold[i]]])
    
    #use for score > 0.7
    #th1 = list(norm_sdg_df4.iloc[i] >= cutoff)
    th2 = list(df2.iloc[i])
    found = np.dot(th1, th2)
    if found > 0:
        match += 1
print("Accuracy Score (sim/doc_len*sdg_len): %.2f"% (match/doc_num*100)+"%")

Accuracy Score (sim): 11.76%
Accuracy Score (sim/doc_len): 11.76%
Accuracy Score (sim/sdg_len): 17.65%
Accuracy Score (sim/doc_len*sdg_len): 17.65%


# Similarity (1-gram) Threshold=0.75

In [94]:
documents = data_lemmatized
sdg_l = sdg_list

score_vector1 = np.zeros((len(documents), len(sdg_l)))
score_vector2 = np.zeros((len(documents), len(sdg_l)))
score_vector3 = np.zeros((len(documents), len(sdg_l)))
score_vector4 = np.zeros((len(documents), len(sdg_l)))

for d, doc in tqdm(enumerate(documents)):
    d_len = len(documents[d])
    #print(d)
    for s, sdg in enumerate(sdg_l):
        s_len = len(sdg_l[s])
        score = 0
        for keyword in sdg:
            for word in doc:
                try:
                    sim = model.similarity(word, keyword)
                    if (sim >= 0.75):
                        score += sim 
                except:
                    pass
        score_vector1[d][s] = score
        score_vector2[d][s] = score / d_len
        score_vector3[d][s] = score / s_len
        score_vector4[d][s] = score / (d_len*s_len)

17it [09:03, 31.99s/it]


In [95]:
import numpy

project_sdg_df1 = pd.DataFrame(data=score_vector1)
project_sdg_df2 = pd.DataFrame(data=score_vector2)
project_sdg_df3 = pd.DataFrame(data=score_vector3)
project_sdg_df4 = pd.DataFrame(data=score_vector4)

columns = ['sdg1', 'sdg2', 'sdg3', 'sdg4', 'sdg5', 'sdg6', 'sdg7', 'sdg8', 'sdg9', 'sdg10', 'sdg11', 'sdg12', 'sdg13', 'sdg14', 'sdg15', 'sdg16', 'sdg17']

project_sdg_df1.columns = columns
project_sdg_df2.columns = columns
project_sdg_df3.columns = columns
project_sdg_df4.columns = columns

#threshold1 = numpy.argsort(project_sdg_df1)['sdg17']
#threshold2 = numpy.argsort(project_sdg_df2)['sdg17']
#threshold3 = numpy.argsort(project_sdg_df3)['sdg17']
#threshold4 = numpy.argsort(project_sdg_df4)['sdg17']

In [96]:
from sklearn import preprocessing
min_max_scaler = preprocessing.MinMaxScaler()
norm_sdg_df1 = pd.DataFrame(min_max_scaler.fit_transform(project_sdg_df1.T).T)
norm_sdg_df2 = pd.DataFrame(min_max_scaler.fit_transform(project_sdg_df2.T).T)
norm_sdg_df3 = pd.DataFrame(min_max_scaler.fit_transform(project_sdg_df3.T).T)
norm_sdg_df4 = pd.DataFrame(min_max_scaler.fit_transform(project_sdg_df4.T).T)

In [107]:
doc_num = 17
i = 0
match = 0
cutoff = 0.8

for i in range(doc_num):
    # use for top-1
    threshold = np.argsort(project_sdg_df1)['sdg17']
    th1 = list(project_sdg_df1.iloc[i] >= project_sdg_df1.iloc[i][columns[threshold[i]]])
    
    #use for score > 0.7
    #th1 = list(norm_sdg_df1.iloc[i] >= cutoff)
    th2 = list(df2.iloc[i])
    found = np.dot(th1, th2)
    if found > 0:
        match += 1
print("Accuracy Score (sim): %.2f"% (match/doc_num*100)+"%")

match=0
for i in range(doc_num):
    # use for top-1
    #threshold = np.argsort(project_sdg_df2)['sdg17']
    #th1 = list(project_sdg_df2.iloc[i] >= project_sdg_df2.iloc[i][columns[threshold[i]]])
    
    #use for score > 0.7
    th1 = list(norm_sdg_df2.iloc[i] >= cutoff)
    th2 = list(df2.iloc[i])
    found = np.dot(th1, th2)
    if found > 0:
        match += 1
    else:
        print (th1)
        print (th2)
print("Accuracy Score (sim/doc_len): %.2f"% (match/doc_num*100)+"%")

match=0
for i in range(doc_num):
    # use for top-1
    #threshold = np.argsort(project_sdg_df3)['sdg17']
    #th1 = list(project_sdg_df3.iloc[i] >= project_sdg_df3.iloc[i][columns[threshold[i]]])
    
    #use for score > 0.7
    th1 = list(norm_sdg_df3.iloc[i] >= cutoff)
    th2 = list(df2.iloc[i])
    found = np.dot(th1, th2)
    if found > 0:
        match += 1
print("Accuracy Score (sim/sdg_len): %.2f"% (match/doc_num*100)+"%")

match=0
for i in range(doc_num):
    # use for top-1
    threshold = np.argsort(project_sdg_df4)['sdg17']
    th1 = list(project_sdg_df4.iloc[i] >= project_sdg_df4.iloc[i][columns[threshold[i]]])
    
    #use for score > 0.7
    #th1 = list(norm_sdg_df4.iloc[i] >= cutoff)
    th2 = list(df2.iloc[i])
    found = np.dot(th1, th2)
    if found > 0:
        match += 1
print("Accuracy Score (sim/doc_len*sdg_len): %.2f"% (match/doc_num*100)+"%")

Accuracy Score (sim): 58.82%
[False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, True, False]
[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[False, False, True, False, False, False, False, True, False, True, True, False, False, False, False, False, False]
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[False, False, True, False, False, False, False, False, True, False, False, True, False, False, False, False, False]
[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Accuracy Score (sim/doc_len): 82.35%
Accuracy Score (sim/sdg_len): 94.12%
Accuracy Score (sim/doc_len*sdg_len): 94.12%


# Similarity (1-gram) Threshold=0.85

In [72]:
documents = data_lemmatized
sdg_l = sdg_list

score_vector1 = np.zeros((len(documents), len(sdg_l)))
score_vector2 = np.zeros((len(documents), len(sdg_l)))
score_vector3 = np.zeros((len(documents), len(sdg_l)))
score_vector4 = np.zeros((len(documents), len(sdg_l)))

for d, doc in tqdm(enumerate(documents)):
    d_len = len(documents[d])
    #print(d)
    for s, sdg in enumerate(sdg_l):
        s_len = len(sdg_l[s])
        score = 0
        for keyword in sdg:
            for word in doc:
                try:
                    sim = model.similarity(word, keyword)
                    if (sim >= 0.85):
                        score += sim 
                except:
                    pass
        score_vector1[d][s] = score
        score_vector2[d][s] = score / d_len
        score_vector3[d][s] = score / s_len
        score_vector4[d][s] = score / (d_len*s_len)

17it [08:32, 30.13s/it]


In [73]:
import numpy

project_sdg_df1 = pd.DataFrame(data=score_vector1)
project_sdg_df2 = pd.DataFrame(data=score_vector2)
project_sdg_df3 = pd.DataFrame(data=score_vector3)
project_sdg_df4 = pd.DataFrame(data=score_vector4)

columns = ['sdg1', 'sdg2', 'sdg3', 'sdg4', 'sdg5', 'sdg6', 'sdg7', 'sdg8', 'sdg9', 'sdg10', 'sdg11', 'sdg12', 'sdg13', 'sdg14', 'sdg15', 'sdg16', 'sdg17']

project_sdg_df1.columns = columns
project_sdg_df2.columns = columns
project_sdg_df3.columns = columns
project_sdg_df4.columns = columns

#threshold1 = numpy.argsort(project_sdg_df1)['sdg17']
#threshold2 = numpy.argsort(project_sdg_df2)['sdg17']
#threshold3 = numpy.argsort(project_sdg_df3)['sdg17']
#threshold4 = numpy.argsort(project_sdg_df4)['sdg17']

In [74]:
from sklearn import preprocessing
min_max_scaler = preprocessing.MinMaxScaler()
norm_sdg_df1 = pd.DataFrame(min_max_scaler.fit_transform(project_sdg_df1.T).T)
norm_sdg_df2 = pd.DataFrame(min_max_scaler.fit_transform(project_sdg_df2.T).T)
norm_sdg_df3 = pd.DataFrame(min_max_scaler.fit_transform(project_sdg_df3.T).T)
norm_sdg_df4 = pd.DataFrame(min_max_scaler.fit_transform(project_sdg_df4.T).T)

In [75]:
doc_num = 17
i = 0
match = 0

for i in range(doc_num):
    # use for top-1
    threshold = np.argsort(project_sdg_df1)['sdg17']
    th1 = list(project_sdg_df1.iloc[i] >= project_sdg_df1.iloc[i][columns[threshold[i]]])
    
    #use for score > 0.7
    #th1 = list(norm_sdg_df1.iloc[i] >= cutoff)
    th2 = list(df2.iloc[i])
    found = np.dot(th1, th2)
    if found > 0:
        match += 1
print("Accuracy Score (sim): %.2f"% (match/doc_num*100)+"%")

match=0
for i in range(doc_num):
    # use for top-1
    threshold = np.argsort(project_sdg_df2)['sdg17']
    th1 = list(project_sdg_df2.iloc[i] >= project_sdg_df2.iloc[i][columns[threshold[i]]])
    
    #use for score > 0.7
    #th1 = list(norm_sdg_df2.iloc[i] >= cutoff)
    th2 = list(df2.iloc[i])
    found = np.dot(th1, th2)
    if found > 0:
        match += 1
print("Accuracy Score (sim/doc_len): %.2f"% (match/doc_num*100)+"%")

match=0
for i in range(doc_num):
    # use for top-1
    threshold = np.argsort(project_sdg_df3)['sdg17']
    th1 = list(project_sdg_df3.iloc[i] >= project_sdg_df3.iloc[i][columns[threshold[i]]])
    
    #use for score > 0.7
    #th1 = list(norm_sdg_df3.iloc[i] >= cutoff)
    th2 = list(df2.iloc[i])
    found = np.dot(th1, th2)
    if found > 0:
        match += 1
print("Accuracy Score (sim/sdg_len): %.2f"% (match/doc_num*100)+"%")

match=0
for i in range(doc_num):
    # use for top-1
    threshold = np.argsort(project_sdg_df4)['sdg17']
    th1 = list(project_sdg_df4.iloc[i] >= project_sdg_df4.iloc[i][columns[threshold[i]]])
    
    #use for score > 0.7
    #th1 = list(norm_sdg_df4.iloc[i] >= cutoff)
    th2 = list(df2.iloc[i])
    found = np.dot(th1, th2)
    if found > 0:
        match += 1
print("Accuracy Score (sim/doc_len*sdg_len): %.2f"% (match/doc_num*100)+"%")

Accuracy Score (sim): 76.47%
Accuracy Score (sim/doc_len): 76.47%
Accuracy Score (sim/sdg_len): 94.12%
Accuracy Score (sim/doc_len*sdg_len): 94.12%


# Similarity (n-gram) Threshold=0.6

In [76]:
#defining all 17 SDGs
sdg1 = [['africa'],['basic', 'services'],['class'],['developing', 'countries'],['disadvantaged'],['economic', 'resources'],['end', 'poverty'],['environment'],['equality'],['extreme', 'poverty'],['financial', 'inclusion'],['income'],['income', 'equality'],['microfinance'],['non-discrimination'],['poor'],['poor', 'and', 'vulnerable'],['poverty'],['poverty', 'eradication'],['poverty', 'line'],['quality', 'of', 'life'],['resources'],['social', 'protection', 'systems'],['social', 'protection'],['sustainable'],['third', 'world'],['vulnerable'],['wealth', 'distribution']]
sdg2 = [['agricultural', 'orientation', 'index'],['agricultural', 'productivity'],['agriculture'],['consume'],['crop', 'diversity'],['crops'],['doha', 'development', 'round'],['doha', 'round'],['end', 'hunger'],['environment'],['food'],['food', 'gap'],['food', 'production'],['food', 'reserves'],['food', 'security'],['genetic', 'diversity'],['genetic', 'diversity', 'of', 'seeds'],['genetics'],['hunger'],['hungry', 'people'],['improved', 'nutrition'],['innovations', 'and', 'health'],['legumes'],['maize'],['malnourished'],['malnutrition'],['nutrition'],['nutritional', 'needs'],['nutritious'],['poverty'],['produce'],['productivity'],['quality', 'of', 'life'],['resilient', 'agriculture'],['rural', 'infrastructure'],['small', 'scale', 'food', 'producers'],['stunted', 'growth'],['stunting'],['sufficient', 'food'],['sustainable'],['sustainable', 'agriculture'],['sustainable', 'food', 'production'],['trade', 'diversity'],['trade', 'restrictions'],['under', 'nourished'],['undernourished'],['wasting'],['world', 'hungry']]
sdg3 = [['access', 'to', 'clean', 'water', 'and', 'sanitation'],['affordable', 'medicines'],['aids'],['air', 'contamination'],['air', 'pollution'],['alcohol', 'abuse'],['antenatal', 'care'],['antiretroviral'],['antiretroviral', 'therapy'],['biomedical'],['bodily', 'autonomy'],['child', 'deaths'],['contraceptive', 'use'],['death', 'rate'],['dental'],['disability', 'and', 'family', 'support'],['disability', 'and', 'inclusion'],['disability', 'and', 'politics', 'of', 'location'],['diseases'],['family', 'planning'],['health'],['health', 'in', 'resource-constrained', 'settings'],['health', 'worker', 'density'],['healthy'],['healthy', 'lives'],['hepatitis'],['hiv'],['improving', 'mortality'],['increasing', 'life', 'expectancy'],['indigenous'],['infected'],['international', 'health', 'policy'],['international', 'health', 'regulations'],['malaria'],['maternal', 'mortality'],['measles'],['medical'],['mental', 'health'],['mortality'],['mortality', 'rate'],['narcotic', 'drug', 'abuse'],['neonatal', 'mortality'],['polio'],['premature', 'mortality'],['preventable', 'deaths'],['reducing', 'malaria'],['reducing', 'mortality'],['refugees', 'and', 'health', 'services'],['reproductive', 'health'],['road', 'traffic', 'accidents'],['sexual', 'and', 'reproductive', 'health-care'],['sexual', 'health'],['soil', 'contamination'],['soil', 'pollution'],['tobacco', 'control'],['treatment', 'of', 'substance', 'abuse'],['tuberculosis'],['universal', 'health'],['universal', 'health', 'coverage'],['vaccines'],['vaccines', 'in', 'developing', 'countries'],['violence'],['wash'],['water', 'sanitation', 'and', 'hygiene', 'for', 'all'],['water', 'borne', 'disease'],['wellbeing'],['well', 'being'],['well-being'],['world', 'health', 'organisation']]
sdg4 = [['access', 'to', 'education'],['basic', 'education'],['basic', 'literacy'],['basic', 'literacy', 'skills'],['cultural', 'diversity'],['disability'],['disability', 'and', 'education'],['early', 'childhood'],['early', 'childhood', 'development'],['education'],['education', 'for', 'sustainability'],['education', 'in', 'developing'],['enrolment'],['equal', 'access'],['equal', 'education'],['equitable', 'education'],['gender', 'disparities', 'in', 'education'],['gender', 'disparity'],['gender', 'equality'],['gender', 'equity'],['gender', 'sensitive'],['global', 'citizenship'],['global', 'education'],['inclusion', 'and', 'education'],['inclusive'],['innovation'],['international', 'cooperation'],['learning', 'opportunities'],['lifelong', 'learning'],['literacy'],['literacy', 'skills'],['numeracy'],['preprimary', 'education'],['primary', 'education'],['qualified', 'teachers'],['refugees', 'and', 'learning'],['scholarships'],['school'],['school', 'enrolment'],['secondary', 'education'],['teacher', 'training'],['universal', 'education'],['vocational', 'training'],['vulnerable'],['women', 'rights']]
sdg5 = [['basic', 'living', 'standards'],['dignity'],['disadvantaged'],['discrimination'],['employment'],['empower', 'girls'],['empowerment'],['empowerment', 'of', 'women'],['empower', 'women'],['women', 'empowerment'],['equal', 'access'],['equal', 'opportunities'],['equality'],['exploitation'],['female', 'genital', 'mutilation'],['feminism'],['forced', 'marriage'],['gender'],['gender', 'discrimination'],['gender', 'equality'],['gender', 'parity'],['governance', 'and', 'gender'],['human', 'rights'],['human', 'trafficking'],['humanitarian'],['marginalised'],['parity'],['pay'],['reproductive', 'rights'],['sexual', 'and', 'reproductive', 'health'],['sexual', 'exploitation'],['sexual', 'violence'],['social', 'inclusion'],['trafficking'],['universal', 'health', 'coverage'],['violence'],['violence', 'against', 'girls'],['violence', 'against', 'women'],['women'],['women', 'in', 'work'],['women', 'rights'],['workplace', 'equality']]
sdg6 = [['accessible', 'water'],['affordable', 'drinking', 'water'],['aquifer'],['cities'],['clean', 'water'],['contaminated'],['defecation'],['desalination'],['diarrhoeal', 'diseases'],['drought'],['dumping'],['ecosystem', 'protection'],['ecosystem', 'restoration'],['equitable', 'sanitation'],['floods'],['fresh', 'water'],['hydropower'],['hygiene'],['improving', 'water'],['inadequate', 'water'],['inadequate', 'water', 'supply'],['infrastructure'],['irrigation'],['lakes'],['latrines'],['open', 'defecation'],['pollution'],['recycled', 'water'],['reuse'],['river', 'basins'],['rivers'],['safe', 'drinking', 'water'],['sanitation'],['sanitation', 'and', 'hygiene'],['sanitation', 'management'],['sewerage'],['sustainable', 'water', 'management'],['sustainable', 'withdrawals'],['third', 'world'],['toilets'],['untreated', 'wastewater'],['urban'],['waste'],['wastewater'],['wastewater', 'treatment'],['water'],['water', 'access'],['water', 'disasters'],['water', 'ecosystems'],['water', 'efficiency'],['water', 'harvesting'],['water', 'quality'],['water', 'resources', 'management'],['water', 'scarcity'],['water', 'supply'],['water-related', 'ecosystems'],['water-use', 'efficiency']]
sdg7 = [['affordable', 'energy'],['alternative', 'energy'],['animal', 'waste'],['battery'],['carbon'],['charcoal'],['clean', 'energy'],['clean', 'energy', 'technology'],['clean', 'fuel', 'technology'],['clean', 'fuels'],['cleaner', 'fossil', 'fuel', 'technology'],['climate', 'goal'],['coal'],['electricity'],['electricity', 'infrastructure'],['emissions'],['energy'],['energy', 'efficiency'],['energy', 'infrastructure'],['energy', 'research'],['energy', 'technology'],['fossil-fuel'],['green', 'economy'],['greenhouse', 'gas'],['greenhouse', 'gas', 'emissions'],['hydroelectric'],['low', 'carbon'],['modern', 'electricity'],['modern', 'energy'],['reliable', 'energy'],['renewable'],['renewable', 'energy'],['renewable', 'power'],['solar'],['solar', 'energy'],['solar', 'power'],['sustainable', 'energy'],['sustainable', 'energy', 'services'],['sustainable', 'power'],['vehicles'],['wave'],['wind'],['wind', 'power'],['wind', 'turbine'],['wood']]
sdg8 = [['aid', 'for', 'trade'],['banking'],['child', 'labour'],['child', 'soldiers'],['creativity', 'and', 'innovation'],['culture'],['decent', 'work'],['decent', 'work', 'for', 'all'],['development', 'oriented', 'policy'],['economic', 'growth'],['economic', 'productivity'],['economy'],['enterprises'],['entrepreneurship'],['equal', 'pay'],['finance'],['financial', 'services'],['forced', 'labour'],['gdp', 'growth'],['global', 'resource', 'efficiency'],['global', 'trade'],['gross', 'domestic', 'product', 'growth'],['human', 'trafficking'],['inclusive', 'economic', 'growth'],['innovation'],['insurance'],['job', 'creation'],['jobs'],['labour', 'market'],['labour', 'rights'],['micro', 'finance'],['migrant', 'workers'],['modern', 'slavery'],['poverty', 'eradication'],['poverty', 'line'],['productive', 'employment'],['productivity'],['public', 'policy'],['quality', 'jobs'],['quality', 'of', 'life'],['resource', 'efficiency'],['safe', 'work'],['secure', 'work'],['slavery'],['social', 'policies'],['society'],['stable', 'employment'],['stable', 'jobs'],['sustainable', 'consumption'],['sustainable', 'economic', 'growth'],['sustainable', 'production'],['sustainable', 'tourism'],['trade'],['unemployment'],['well-paid', 'jobs'],['women', 'migrants'],['work'],['work', 'opportunities'],['world', 'trade'],['youth', 'employment'],['youth', 'unemployment']]
sdg9 = [['access', 'to', 'the', 'internet'],['affordable', 'access'],['affordable', 'credit'],['clean', 'technologies'],['cooperation'],['data', 'banks'],['economic', 'development'],['electrical', 'power'],['energy'],['enterprises'],['environmentally', 'sound', 'technologies'],['financial', 'services'],['ict', 'infrastructure'],['industrial', 'diversification'],['industrialisation'],['information', 'and', 'communication', 'technology'],['infrastructure'],['innovation'],['internet', 'access'],['irrigation'],['knowledge', 'in', 'education', 'for', 'all'],['mobile', 'networks', 'in', 'developing', 'countries'],['national', 'security'],['network', 'infrastructure'],['phone', 'service'],['public', 'policy'],['quality', 'of', 'life'],['regional', 'infrastructure'],['research'],['resilient', 'infrastructure'],['resource', 'use', 'efficiency'],['roads'],['sanitation'],['scientific', 'research'],['society'],['sustainable', 'industrialisation'],['sustainable', 'infrastructure'],['technological', 'capabilities'],['technology'],['trade'],['transborder', 'infrastructure'],['transport'],['value', 'chains'],['value', 'chains', 'and', 'markets'],['water', 'infrastructure'],['water', 'resources']]
sdg10 = [['affordable', 'housing'],['age'],['ageism'],['business'],['children'],['culture'],['developing', 'countries'],['developing', 'states'],['development', 'assistance'],['disabilities'],['discrimination'],['discriminatory'],['economy'],['education'],['empower'],['equal', 'opportunity'],['equality'],['equity'],['ethnicity'],['financial', 'assistance'],['foreign', 'aid'],['foreign', 'investment'],['gender'],['global', 'financial', 'markets'],['health'],['homelessness'],['homophobia'],['human', 'rights'],['inclusion'],['income', 'growth'],['income', 'inequality'],['indigenous'],['inequalities'],['inequality'],['migrant', 'remittance'],['migration'],['population', 'growth'],['poverty'],['public', 'policy'],['quality', 'of', 'life'],['race'],['racisim'],['reduce', 'inequalities'],['religion'],['rural'],['sex'],['sexism'],['social', 'protection'],['society'],['vulnerable', 'nations'],['world', 'trade']]
sdg11 = [['adaptable'],['adaptation'],['affordable', 'housing'],['air', 'pollution'],['air', 'quality'],['cities'],['climate', 'change'],['community'],['cultural', 'heritage'],['decentralisation'],['development', 'planning'],['disaster', 'management'],['disaster', 'risk', 'reduction'],['disaster', 'strategy'],['disasters'],['fine', 'particulate', 'matter'],['green', 'spaces'],['heritage'],['housing'],['human', 'settlements'],['impact', '', 'of', 'cities'],['inadequate', 'housing'],['informal', 'settlements'],['infrastructure'],['land'],['land', 'consumption'],['local', 'materials'],['mitigation'],['natural', 'disasters'],['natural', 'heritage'],['over', 'crowding'],['pollution'],['population'],['population', 'growth'],['public', 'spaces'],['public', 'transport'],['resilient'],['resilient', 'buildings'],['resource', 'efficiency'],['resource', 'needs'],['risk', 'reduction', 'strategy'],['road', 'safety'],['safe', 'cities'],['shanty'],['slums'],['smart', 'cities'],['solid', 'waste'],['suburban'],['sustainable'],['sustainable', 'building'],['sustainable', 'buildings'],['sustainable', 'cities/city'],['sustainable', 'communities'],['sustainable', 'urbanisation'],['town', 'planning'],['transport'],['transport', 'systems'],['urban'],['urban', 'development'],['urban', 'planning'],['urban', 'sustainability'],['urbanisation'],['waste'],['waste', 'generation'],['waste', 'management'],['water'],['water', 'related', 'disasters']]
sdg12 = [['capitalism'],['cars'],['circular', 'economy'],['commercial', 'enterprises'],['consumer', 'levels'],['consumerism'],['consumption'],['deep', 'decarbonisation'],['ecological'],['efficient', 'use', 'of', 'resources'],['energy'],['energy', 'consumption'],['energy', 'efficiency'],['energy', 'use'],['food'],['food', 'losses'],['food', 'supply'],['food', 'waste'],['fossil', 'fuel', 'subsidies'],['future', 'proof'],['global', 'food', 'waste'],['greenhouse', 'gasses'],['harvest', 'losses'],['life', 'cycle'],['market', 'distortions'],['materialism'],['materials', 'goods'],['monitoring', 'sustainable', 'development'],['natural', 'resources'],['obsolescence'],['overconsumption'],['production'],['recycle'],['recycling'],['reduce', 'waste', 'generation'],['reduction'],['renewable'],['resource', 'efficiency'],['responsible', 'production', 'chains'],['retail'],['retail', 'industry'],['reuse'],['sustainable'],['sustainable', 'consumption'],['sustainable', 'management'],['sustainable', 'practices'],['sustainable', 'production'],['sustainable', 'public', 'procurement'],['sustainable', 'resource', 'use'],['sustainable', 'supply', 'chain'],['sustainable', 'tourism'],['vehicles'],['waste'],['wasteful', 'consumption'],['water'],['water', 'pollution'],['water', 'supply']]
sdg13 = [['average', 'global', 'temperature'],['carbon'],['carbon', 'dioxide'],['changing', 'weather', 'patterns'],['climate'],['climate', 'action'],['climate', 'adaptation'],['climate', 'and', 'gender'],['climate', 'and', 'infectious', 'disease'],['climate', 'and', 'politics'],['climate', 'change'],['climate', 'change', 'management'],['climate', 'change', 'planning'],['climate', 'change', 'policy'],['climate', 'early', 'warning'],['climate', 'hazards'],['climate', 'impact'],['climate', 'mitigation'],['climate', 'refugees'],['climate', 'related', 'hazards'],['climate', 'resilience'],['co2', 'capture'],['co2', 'conversion'],['cop', '21'],['cop', '22'],['ecosystems'],['emissions'],['extreme', 'weather'],['extreme', 'weather', 'events'],['global', 'mean', 'temperature'],['global', 'temperature'],['global', 'warming'],['greenhouse', 'gas'],['greenhouse', 'gas', 'emissions'],['greenhouse', 'gases'],['ice', 'loss'],['low-carbon', 'economy'],['natural', 'disasters'],['natural', 'systems'],['ocean', 'warming'],['paris', 'agreement'],['pollution'],['renewable'],['sea', 'level', 'rise'],['rising', 'sea'],['rising', 'sea', 'level'],['temperature'],['warming']]
sdg14 = [['artisanal', 'fishers'],['biodiversity'],['carbon', 'dioxide'],['coastal', 'biodiversity'],['coastal', 'ecosystems'],['coastal', 'habitats'],['coastal', 'parks'],['coastal', 'resources'],['coastlines'],['conserve'],['conserve', 'oceans'],['coral', 'bleaching'],['coral', 'reef'],['ecosystem', 'management'],['fish', 'species'],['fish', 'stocks'],['fish', 'stocks', 'and', 'fisheries', 'management'],['fisheries'],['fishers'],['fishing'],['fishing', 'practices'],['global', 'warming'],['illegal', 'fishing'],['kelp'],['law', 'of', 'the', 'sea'],['marine'],['marine', 'areas'],['marine', 'biodiversity'],['marine', 'ecosystems'],['marine', 'fisheries'],['marine', 'parks'],['marine', 'pollution'],['marine', 'resources'],['ocean'],['ocean', 'acidification'],['ocean', 'temperature'],['oceanography'],['oceans'],['overfishing'],['productive', 'oceans'],['protected', 'areas'],['sea', 'grasses'],['seas'],['sustainable', 'ecosystems'],['unregulated', 'fishing'],['water', 'resources', 'and', 'policy']]
sdg15 = [['afforestation'],['agriculture'],['animals'],['arable', 'land'],['bees'],['biodiversity'],['biodiversity', 'loss'],['conservation'],['deforestation'],['desertification'],['drought'],['drylands'],['ecosystem'],['ecosystem', 'restoration'],['ecosystems'],['extinct'],['extinct', 'species'],['extinction'],['forest'],['forest', 'management'],['forests'],['genetic', 'resources'],['illegal', 'wildlife', 'products'],['illicit', 'trafficking'],['indigenous'],['indigenous', 'populations'],['invasive', 'alien', 'species'],['land', 'conservation'],['land', 'degradation'],['land', 'loss'],['land', 'use', 'and', 'sustainability'],['manage', 'forests'],['managed', 'forests'],['micro-organisms'],['permaculture'],['plants'],['poaching'],['poverty'],['protected', 'fauna'],['protected', 'flora'],['protected', 'species'],['reforestation'],['soil'],['soil', 'degradation'],['species'],['strategic', 'plan', 'for', 'biodiversity'],['terrestrial', 'ecosystems'],['threatened', 'species'],['tree'],['tree', 'species'],['wetlands']]
sdg16 = [['abuse'],['accountability'],['accountable', 'institutions'],['arbitrary', 'detention'],['arms'],['arms', 'trafficking'],['birth', 'registration'],['bribery'],['combat', 'terrorism'],['conflict', 'resolution'],['conflicts'],['corruption'],['discrimination'],['education'],['enforced', 'disappearance'],['equal', 'access'],['equity'],['exploitation'],['flow', 'of', 'arms'],['freedom'],['geography', 'of', 'poverty'],['governance'],['hate', 'crime'],['human', 'rights'],['human', 'trafficking'],['illegal', 'arms'],['illicit', 'financial', 'flows'],['inclusion'],['inclusive', 'institutions'],['inclusive', 'societies/society'],['institutions'],['internally', 'displaced'],['judiciary'],['justice'],['justice', 'for', 'all'],['legal', 'identity'],['national', 'security'],['non-violence'],['organized', 'crime'],['paris', 'principles'],['peace'],['peaceful', 'societies'],['physical', 'abuse'],['police'],['prevent', 'violence'],['psychological', 'abuse'],['public', 'policy'],['quality', 'of', 'life'],['representative', 'decision-making'],['rule', 'of', 'law'],['security', 'threats'],['sexual', 'abuse'],['sexual', 'violence'],['stolen', 'assets'],['tax', 'evasion'],['theft'],['torture'],['trafficking'],['transparency'],['unsentenced', 'detainees'],['unstable', 'societies'],['victims', 'of', 'violence'],['violence'],['violence', 'against', 'women', 'and', 'children'],['violence', 'rates'],['weapon', 'seizures']]
sdg17 = [['capacity', 'building'],['civil', 'society', 'partnerships'],['communication', 'technologies'],['debt', 'sustainability'],['development', 'assistance'],['disaggregated', 'data'],['doha', 'development', 'agenda'],['entrepreneurship'],['environmentally', 'sound', 'technologies'],['foreign', 'direct', 'investments'],['fostering', 'innovation'],['free', 'trade'],['fundamental', 'principles', 'of', 'official', 'statistics'],['global', 'partnership'],['global', 'partnership', 'for', 'sustainable', 'development'],['global', 'stability'],['international', 'aid'],['international', 'cooperation'],['international', 'population', 'and', 'housing', 'census'],['international', 'support'],['international', 'support', 'for', 'developing', 'countries'],['knowledge', 'sharing'],['multi-stakeholder', 'partnerships'],['poverty', 'eradication'],['public-private', 'partnerships'],['science', 'cooperation', 'agreements'],['technology', 'cooperation', 'agreements'],['technology', 'transfer'],['weighted', 'tariff', 'average'],['women', 'entrepreneurs'],['world', 'trade', 'organization']]
sdg_list = [sdg1, sdg2, sdg3, sdg4, sdg5, sdg6, sdg7, sdg8, sdg9, sdg10, sdg11, sdg12, sdg13, sdg14, sdg15, sdg16, sdg17]

In [77]:
documents = data_lemmatized
sdg_l = sdg_list

score_vector1 = np.zeros((len(documents), len(sdg_l)))
score_vector2 = np.zeros((len(documents), len(sdg_l)))
score_vector3 = np.zeros((len(documents), len(sdg_l)))
score_vector4 = np.zeros((len(documents), len(sdg_l)))

for d, doc in tqdm(enumerate(documents)):
    d_len = len(documents[d])
    #print(d)
    for s, sdg in enumerate(sdg_l):
        s_len = len(sdg_l[s])
        score = 0
        for keyword in sdg:
            for word in doc:
                a = list()
                a.append(word)
                try:
                    sim = model.n_similarity(a, keyword)
                    if (sim >= 0.6):
                        score += sim 
                except:
                    pass
        score_vector1[d][s] = score
        score_vector2[d][s] = score / d_len
        score_vector3[d][s] = score / s_len
        score_vector4[d][s] = score / (d_len*s_len)

17it [24:29, 86.46s/it]


In [78]:
import numpy

project_sdg_df1 = pd.DataFrame(data=score_vector1)
project_sdg_df2 = pd.DataFrame(data=score_vector2)
project_sdg_df3 = pd.DataFrame(data=score_vector3)
project_sdg_df4 = pd.DataFrame(data=score_vector4)

columns = ['sdg1', 'sdg2', 'sdg3', 'sdg4', 'sdg5', 'sdg6', 'sdg7', 'sdg8', 'sdg9', 'sdg10', 'sdg11', 'sdg12', 'sdg13', 'sdg14', 'sdg15', 'sdg16', 'sdg17']

project_sdg_df1.columns = columns
project_sdg_df2.columns = columns
project_sdg_df3.columns = columns
project_sdg_df4.columns = columns

#threshold1 = numpy.argsort(project_sdg_df1)['sdg17']
#threshold2 = numpy.argsort(project_sdg_df2)['sdg17']
#threshold3 = numpy.argsort(project_sdg_df3)['sdg17']
#threshold4 = numpy.argsort(project_sdg_df4)['sdg17']

In [79]:
from sklearn import preprocessing
min_max_scaler = preprocessing.MinMaxScaler()
norm_sdg_df1 = pd.DataFrame(min_max_scaler.fit_transform(project_sdg_df1.T).T)
norm_sdg_df2 = pd.DataFrame(min_max_scaler.fit_transform(project_sdg_df2.T).T)
norm_sdg_df3 = pd.DataFrame(min_max_scaler.fit_transform(project_sdg_df3.T).T)
norm_sdg_df4 = pd.DataFrame(min_max_scaler.fit_transform(project_sdg_df4.T).T)

In [80]:
doc_num = 17
i = 0
match = 0

for i in range(doc_num):
    # use for top-1
    threshold = np.argsort(project_sdg_df1)['sdg17']
    th1 = list(project_sdg_df1.iloc[i] >= project_sdg_df1.iloc[i][columns[threshold[i]]])
    
    #use for score > 0.7
    #th1 = list(norm_sdg_df1.iloc[i] >= cutoff)
    th2 = list(df2.iloc[i])
    found = np.dot(th1, th2)
    if found > 0:
        match += 1
print("Accuracy Score (sim): %.2f"% (match/doc_num*100)+"%")

match=0
for i in range(doc_num):
    # use for top-1
    threshold = np.argsort(project_sdg_df2)['sdg17']
    th1 = list(project_sdg_df2.iloc[i] >= project_sdg_df2.iloc[i][columns[threshold[i]]])
    
    #use for score > 0.7
    #th1 = list(norm_sdg_df2.iloc[i] >= cutoff)
    th2 = list(df2.iloc[i])
    found = np.dot(th1, th2)
    if found > 0:
        match += 1
print("Accuracy Score (sim/doc_len): %.2f"% (match/doc_num*100)+"%")

match=0
for i in range(doc_num):
    # use for top-1
    threshold = np.argsort(project_sdg_df3)['sdg17']
    th1 = list(project_sdg_df3.iloc[i] >= project_sdg_df3.iloc[i][columns[threshold[i]]])
    
    #use for score > 0.7
    #th1 = list(norm_sdg_df3.iloc[i] >= cutoff)
    th2 = list(df2.iloc[i])
    found = np.dot(th1, th2)
    if found > 0:
        match += 1
print("Accuracy Score (sim/sdg_len): %.2f"% (match/doc_num*100)+"%")

match=0
for i in range(doc_num):
    # use for top-1
    threshold = np.argsort(project_sdg_df4)['sdg17']
    th1 = list(project_sdg_df4.iloc[i] >= project_sdg_df4.iloc[i][columns[threshold[i]]])
    
    #use for score > 0.7
    #th1 = list(norm_sdg_df4.iloc[i] >= cutoff)
    th2 = list(df2.iloc[i])
    found = np.dot(th1, th2)
    if found > 0:
        match += 1
print("Accuracy Score (sim/doc_len*sdg_len): %.2f"% (match/doc_num*100)+"%")

Accuracy Score (sim): 11.76%
Accuracy Score (sim/doc_len): 11.76%
Accuracy Score (sim/sdg_len): 17.65%
Accuracy Score (sim/doc_len*sdg_len): 17.65%


# Similarity (n-gram) Threshold=0.85

In [81]:
documents = data_lemmatized
sdg_l = sdg_list

score_vector1 = np.zeros((len(documents), len(sdg_l)))
score_vector2 = np.zeros((len(documents), len(sdg_l)))
score_vector3 = np.zeros((len(documents), len(sdg_l)))
score_vector4 = np.zeros((len(documents), len(sdg_l)))

for d, doc in tqdm(enumerate(documents)):
    d_len = len(documents[d])
    #print(d)
    for s, sdg in enumerate(sdg_l):
        s_len = len(sdg_l[s])
        score = 0
        for keyword in sdg:
            for word in doc:
                a = list()
                a.append(word)
                try:
                    sim = model.n_similarity(a, keyword)
                    if (sim >= 0.85):
                        score += sim 
                except:
                    pass
        score_vector1[d][s] = score
        score_vector2[d][s] = score / d_len
        score_vector3[d][s] = score / s_len
        score_vector4[d][s] = score / (d_len*s_len)

17it [22:58, 81.10s/it]


In [82]:
import numpy

project_sdg_df1 = pd.DataFrame(data=score_vector1)
project_sdg_df2 = pd.DataFrame(data=score_vector2)
project_sdg_df3 = pd.DataFrame(data=score_vector3)
project_sdg_df4 = pd.DataFrame(data=score_vector4)

columns = ['sdg1', 'sdg2', 'sdg3', 'sdg4', 'sdg5', 'sdg6', 'sdg7', 'sdg8', 'sdg9', 'sdg10', 'sdg11', 'sdg12', 'sdg13', 'sdg14', 'sdg15', 'sdg16', 'sdg17']

project_sdg_df1.columns = columns
project_sdg_df2.columns = columns
project_sdg_df3.columns = columns
project_sdg_df4.columns = columns

#threshold1 = numpy.argsort(project_sdg_df1)['sdg17']
#threshold2 = numpy.argsort(project_sdg_df2)['sdg17']
#threshold3 = numpy.argsort(project_sdg_df3)['sdg17']
#threshold4 = numpy.argsort(project_sdg_df4)['sdg17']

In [83]:
from sklearn import preprocessing
min_max_scaler = preprocessing.MinMaxScaler()
norm_sdg_df1 = pd.DataFrame(min_max_scaler.fit_transform(project_sdg_df1.T).T)
norm_sdg_df2 = pd.DataFrame(min_max_scaler.fit_transform(project_sdg_df2.T).T)
norm_sdg_df3 = pd.DataFrame(min_max_scaler.fit_transform(project_sdg_df3.T).T)
norm_sdg_df4 = pd.DataFrame(min_max_scaler.fit_transform(project_sdg_df4.T).T)

In [84]:
doc_num = 17
i = 0
match = 0

for i in range(doc_num):
    # use for top-1
    threshold = np.argsort(project_sdg_df1)['sdg17']
    th1 = list(project_sdg_df1.iloc[i] >= project_sdg_df1.iloc[i][columns[threshold[i]]])
    
    #use for score > 0.7
    #th1 = list(norm_sdg_df1.iloc[i] >= cutoff)
    th2 = list(df2.iloc[i])
    found = np.dot(th1, th2)
    if found > 0:
        match += 1
print("Accuracy Score (sim): %.2f"% (match/doc_num*100)+"%")

match=0
for i in range(doc_num):
    # use for top-1
    threshold = np.argsort(project_sdg_df2)['sdg17']
    th1 = list(project_sdg_df2.iloc[i] >= project_sdg_df2.iloc[i][columns[threshold[i]]])
    
    #use for score > 0.7
    #th1 = list(norm_sdg_df2.iloc[i] >= cutoff)
    th2 = list(df2.iloc[i])
    found = np.dot(th1, th2)
    if found > 0:
        match += 1
print("Accuracy Score (sim/doc_len): %.2f"% (match/doc_num*100)+"%")

match=0
for i in range(doc_num):
    # use for top-1
    threshold = np.argsort(project_sdg_df3)['sdg17']
    th1 = list(project_sdg_df3.iloc[i] >= project_sdg_df3.iloc[i][columns[threshold[i]]])
    
    #use for score > 0.7
    #th1 = list(norm_sdg_df3.iloc[i] >= cutoff)
    th2 = list(df2.iloc[i])
    found = np.dot(th1, th2)
    if found > 0:
        match += 1
print("Accuracy Score (sim/sdg_len): %.2f"% (match/doc_num*100)+"%")

match=0
for i in range(doc_num):
    # use for top-1
    threshold = np.argsort(project_sdg_df4)['sdg17']
    th1 = list(project_sdg_df4.iloc[i] >= project_sdg_df4.iloc[i][columns[threshold[i]]])
    
    #use for score > 0.7
    #th1 = list(norm_sdg_df4.iloc[i] >= cutoff)
    th2 = list(df2.iloc[i])
    found = np.dot(th1, th2)
    if found > 0:
        match += 1
print("Accuracy Score (sim/doc_len*sdg_len): %.2f"% (match/doc_num*100)+"%")

Accuracy Score (sim): 70.59%
Accuracy Score (sim/doc_len): 70.59%
Accuracy Score (sim/sdg_len): 94.12%
Accuracy Score (sim/doc_len*sdg_len): 94.12%


In [85]:
th1

[False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 False]

In [87]:
df2

,sdg1,sdg2,sdg3,sdg4,sdg5,sdg6,sdg7,sdg8,sdg9,sdg10,sdg11,sdg12,sdg13,sdg14,sdg15,sdg16,sdg17
0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
3,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
7,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0


In [92]:
np.argsort(project_sdg_df1)['sdg17']

0     10
1      9
2     10
3      2
4     11
5      7
6     15
7     15
8      8
9     13
10     7
11     5
12    10
13    14
14     6
15     3
16    11
Name: sdg17, dtype: int64